# Kaggle Challenge Tennis : 

### Timothée Poulain M2 DAC, UPMC, FDMS

#### Pseudo de soumission : 'Djoko_oula'

$\textbf{Select Data:}$ Integrate,collect data.    
$\textbf{Preprocess Data:}$ Clean it   
$\textbf{Transform Data:}$  Feature Engineer  
$\textbf{Model Data:}$ Create models, evaluate them and tune them.



### But de ce Notebook : 

Ce Notebook correspond à un challenge de prédiction des matchs de tennis pour le cours FDMS.     
Nous devons analyser et prédire l'issue des matchs de tennis pour l'année 2016.     
Nous sommes en possession d'une database de 85000 matchs durant la période 2012-2016.



### Base de données utilisés : 

- "games_atp" : base d'entrainement de 85000 matchs avec résultats
- "games_atp_public" : base de test de 7500 matchs sans résultats
- "facts_atp" : base de différentes statistique sur les matchs (doubles fautes, aces,...)
- "players_atp" : base de différentes informations relatives aux joueurs (nationnalité, ...)
- "rounds" : base permettant d'identifier à quel stade du tournoi le joueur est. (8ème, quart,...)
- "tours_atp": base décrivant les différents tournois. 
- "ratings_atp" : base permettant d'avoir le classement des joueurs pour chaque semaine
- "courts" : base permettant d'associer le type de surface de jeu

### Librairies utilisées :

- Pandas 
- Numpy
- Matplotlib
- Scikit-Learn


### Data Handling : 

$\textbf{-> Importer et Loader données via pandas}$    
$\textbf{-> Clean data}$    
_Parse Date (Year, Month, Day)    
_Parse Date (WEEK) pour la table "rating"    
_Drop outliers pour la table "rating"     
$\textbf{-> DataViz}$

### Data Analysis : 

Dans le deuxième ipython notebook, vous trouvez les différentes techniques utilisées

$\textbf{Supervised Machine learning :}$    
_Random Forest   
_Gradient Boosting    
_Tune parameters   
_RFECV (features selection)  
    

### Features Handling : 

#### Step 1 : 9 features

$\textbf{Utilisation des features de départ de la base 'games_atp'}$
- On parse les données des résultats (1 ou 2)
- On parse les données temporelles en 3 colonnes (YEAR, MONTH, DAY )
On obtient alors 8 features de départs :          
(Identifiant_J1, Identifiant_J2, YEAR, MONTH, DAY, RESULT, ID_TOURNOI, ID_ROUND)



#### Step 2 : 12 features

$\textbf{Créations des Features du classements des joueurs suivant chaque semaines en terme de rang et de points}$
- Création d'une colonne DATE_G_2 permettant de merger les classements hebdomadaires des joueurs. 
- Si le classement du joueurs n'est pas inscrit dans la base de données 'ratings', j'utilise le classement le plus récent.
- Suppression d'outliers de la base 'ratings'. Quelques joueurs disposent de deux classements la même semaine

#### Step 3 : 14 features

$\textbf{Création des Features relative aux % de victoires sur l'ensemble des matchs depuis 2012}$
- Création d'une colonne 'WIN_POUR_1' relative aux pourcentages de victoire du J1
- Création d'une colonne 'WIN_POUR_2' relative aux poucentages de victoire du J2

#### Step 4 : 17 features 

$\textbf{Création des Features relatives aux % de victoires pour chaque type de courts}$
- Création d'une feature 'WIN_FOR_COURT_1' relative aux pourcentages de victoire du J1 pour ce type de court
- Création d'une feature 'WIN_FOR_COURT_2' relative aux pourcentages de victoire du J2 pour ce type de court

#### Step 4 : 19 features 

$\textbf{Création des Features relatives aux % de victoires pour chaque tournois}$
- Chaque tournoi étant identifié individuellement, création d'une feature 'ID_T_REAL' permettant de relier les même noms des tournois
- Création d'une feature 'WIN_FOR_TOURNEY_1' relative aux pourcentages de victoire du J1 pour ce tournoi
- Création d'une feature 'WIN_FOR_TOURNEY_2' relative aux pourcentages de victoire du J2 pour ce tournoi

#### Step 5 : 21 Features 

$\textbf{Création des Features relatives à l'age des joueurs}$
- En fonction de l'année, création d'une feature 'AGE_REAL_1' pour le J1
- Et, pour le J2, création d'une feature 'AGE_REAL_2' 

#### Step 6 : 23 Features 

$\textbf{Création des Features relatives aux nombres de victoire entre deux joueurs }$
- Création d'une feature 'FACE_TO_1' relative au nombre de fois que le J1 a gagné contre le J2
- Création d'une feature 'FACE_TO_2' relative au nombre de fois que le J2 a gagné contre le J1

#### Step 7 : 26 Features 

$\textbf{Création des Features relatives à la nationalité des joueurs et à la localisation géographique du Tournoi }$
- Création d'une feature 'COUNTRY_1' relative à la nationnalité du J1
- Création d'une feature 'COUNTRY_1' relative à la nationnalité du J2
- Création d'une feature 'COUNTRY_T' relative à la localisation géographique du tournoi



#### Step 8 : Drop Features 

Différents tests de construction de datasets en enlevant certaines features.

### RESULTAT FINAUX : 

Ils sont disponibles sous forme de diagramme en bas de page du deuxième Ipython.

## Load Library 

In [1]:
import pandas as pd
import numpy as np
import os
from collections import defaultdict
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import cross_val_score

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import (RandomForestClassifier, ExtraTreesClassifier,
                              AdaBoostClassifier)

from sklearn.tree import DecisionTreeClassifier

from sklearn.linear_model import LogisticRegression
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from numpy.linalg import inv
import random
import time
%matplotlib inline 


### Loading File & Create Dictionnaire

In [2]:
dict_table = defaultdict(dict)
path=r'/home/poulain/Documents/Cours DAC/FDMS/Kaggle/load_csv'
for element in os.listdir(path):
    path_absolute=path+"/"+element
    print(path_absolute)
    dict_table[element.split(".")[0]] = pd.read_csv(path_absolute)

/home/poulain/Documents/Cours DAC/FDMS/Kaggle/load_csv/courts.csv
/home/poulain/Documents/Cours DAC/FDMS/Kaggle/load_csv/ratings_atp.csv
/home/poulain/Documents/Cours DAC/FDMS/Kaggle/load_csv/games_atp.csv
/home/poulain/Documents/Cours DAC/FDMS/Kaggle/load_csv/tours_atp.csv
/home/poulain/Documents/Cours DAC/FDMS/Kaggle/load_csv/games_atp_public.csv
/home/poulain/Documents/Cours DAC/FDMS/Kaggle/load_csv/players_atp.csv
/home/poulain/Documents/Cours DAC/FDMS/Kaggle/load_csv/facts_atp.csv
/home/poulain/Documents/Cours DAC/FDMS/Kaggle/load_csv/stat_atp_2.csv
/home/poulain/Documents/Cours DAC/FDMS/Kaggle/load_csv/rounds.csv


## Parse & Shuffle Fonction 

Fonction : 

In [3]:
def shuffle_players(row):
    choice = np.random.choice(2)
    if(choice==1):
        tmp = row["ID1_G"]
        row["ID1_G"] = row["ID2_G"]
        row["ID2_G"] = tmp
        row["RESULT_G"] = 2
    else:
        row["RESULT_G"] = 1
    return row

Fonction parse date

In [4]:
def parsedate_atp(row):
    split_one=row['DATE_G'].split(' ')
    datetime=split_one[0].split('-')
    row['YEAR']=datetime[0]
    row['MONTH']=datetime[1]
    row['DAY']=datetime[2]
    return row

In [5]:
def parsedate_atp_2(row):
    split_one=row['DATE_R'].split(' ')
    datetime=split_one[0].split('-')
    row['YEAR']=datetime[0]
    row['MONTH']=datetime[1]
    row['DAY']=datetime[2]
    return row

In [ ]:
# Parser en Semaine 

In [6]:
dict_table['ratings_atp']
dict_table['ratings_atp']['DATE_R_2']= pd.to_datetime(dict_table['ratings_atp']['DATE_R'])
dict_table['ratings_atp']['DATE_R_2']=dict_table['ratings_atp']['DATE_R_2'].dt.week
dict_table['ratings_atp']

dict_table['games_atp']
dict_table['games_atp']['DATE_G_2']= pd.to_datetime(dict_table['games_atp']['DATE_G'])
dict_table['games_atp']['DATE_G_2']=dict_table['games_atp']['DATE_G_2'].dt.week


In [7]:
def Create_data_toevaluate(X_train,n_X_train,Y_train,n_Y_train,X_test,n_X_test,Y_test,n_Y_test):
    if(n_X_train !=' '):
        X_train.to_csv("train_test_csv3/"+n_X_train+".csv",header=None,index=None)
    if(n_Y_train !=' '):
        Y_train.to_csv("train_test_csv3/"+n_Y_train +".csv",header=None,index=None)
    if(n_X_test !=' '):
        X_test.to_csv("train_test_csv3/"+n_X_test+".csv",header=None,index=None)
    if(n_Y_test !=' '):
        Y_test.to_csv("train_test_csv3/"+n_Y_test+".csv",header=None,index=None)

#### Shuffle and Parse date 

In [8]:
train = dict_table["games_atp"]
train = train.apply(parsedate_atp,axis=1)
train=train.apply(shuffle_players,axis=1)
Y_train = train["RESULT_G"]
X_train= train.drop(["RESULT_G"],axis=1)
X_train=X_train.drop(['DATE_G'],axis=1)
print(len(Y_train))

84485


#### Create train test split

In [ ]:
tmp=len(X_train.YEAR[X_train.YEAR!='2016'])
X_test_final=X_train[tmp+1:len(X_train)]
X_train_final=X_train[0:tmp]
Y_test_final=Y_train[tmp+1:len(Y_train)]
Y_train_final=Y_train[0:tmp]

#### Vérification des données 

In [ ]:
print(len(Y_train_final))
print(len(X_train_final))
print(len(Y_test_final))
print(len(X_test_final))
Y_train=Y_train_final
Y_test=Y_test_final
X_train=X_train_final
X_test=X_test_final


In [9]:
test = dict_table["games_atp_public"]
test = test.apply(parsedate_atp,axis=1)
test=test.drop(["DATE_G"],axis=1)
X_test_csv=test

# Merging between rating_atp_pos_and_games_atp

#### Train pass : 

In [ ]:
#df_rating_merge[df_rating_merge.ID_P_R==5992]

In [10]:
df_rating_merge=dict_table['ratings_atp']
df_rating_merge= df_rating_merge.apply(parsedate_atp_2,axis=1)
df_rating_merge=df_rating_merge.drop(["DATE_R"],axis=1)
df_rating_merge=df_rating_merge.fillna(-9999)
df_rating_merge

,ID_P_R,POINT_R,POS_R,DATE_R_2,YEAR,MONTH,DAY
0,5992,13630,1,1,2012,01,02
1,677,9595,2,1,2012,01,02
2,19,8170,3,1,2012,01,02
3,1075,7380,4,1,2012,01,02
4,673,4925,5,1,2012,01,02
5,3985,4335,6,1,2012,01,02
6,831,3700,7,1,2012,01,02
7,553,2965,8,1,2012,01,02
8,728,2595,9,1,2012,01,02
9,4045,2380,10,1,2012,01,02


In [11]:
dict_table['ratings_atp']
df_merged_first=dict_table['games_atp']
df_merged_first=df_merged_first.apply(shuffle_players,axis=1)
df_merged_first= df_merged_first.apply(parsedate_atp,axis=1)
df_merged_first=df_merged_first.drop(["DATE_G"],axis=1)
df_merged_first=df_merged_first.fillna(-9999)
df_merged_first


,ID1_G,ID2_G,ID_T_G,ID_R_G,RESULT_G,DATE_G_2,YEAR,MONTH,DAY
0,17613,18854,8887,4,1,5,2012,01,31
1,25191,25192,8888,4,2,5,2012,01,31
2,918,10828,8957,4,1,5,2012,01,31
3,3316,2379,8957,4,1,5,2012,01,31
4,7869,6277,8957,4,1,5,2012,01,31
5,7136,2107,8957,4,2,5,2012,01,31
6,72,678,8887,4,2,5,2012,01,31
7,13796,563,8887,4,2,5,2012,01,31
8,775,745,8888,4,2,5,2012,01,31
9,75,1266,8888,4,1,5,2012,01,31


In [12]:
df_merged_first['YEAR']=df_merged_first['YEAR'].astype('int')
df_merged_first['MONTH']=df_merged_first['MONTH'].astype('int')
df_merged_first['DAY']=df_merged_first['DAY'].astype('int')

df_rating_merge['YEAR']=df_rating_merge['YEAR'].astype('int')
df_rating_merge['MONTH']=df_rating_merge['MONTH'].astype('int')
df_rating_merge['DAY']=df_rating_merge['DAY'].astype('int')

df_rating_merge

,ID_P_R,POINT_R,POS_R,DATE_R_2,YEAR,MONTH,DAY
0,5992,13630,1,1,2012,1,2
1,677,9595,2,1,2012,1,2
2,19,8170,3,1,2012,1,2
3,1075,7380,4,1,2012,1,2
4,673,4925,5,1,2012,1,2
5,3985,4335,6,1,2012,1,2
6,831,3700,7,1,2012,1,2
7,553,2965,8,1,2012,1,2
8,728,2595,9,1,2012,1,2
9,4045,2380,10,1,2012,1,2


In [13]:
def remove_outliers_pos(table_old,table_new):
    for i in range(len(table_new)):
        k=table_new[np.where(table_new.index==i,True,False)]['ID1_G'].values[0]
        f=table_old[np.where(table_old.index==i,True,False)]['ID1_G'].values[0]
        if(k!=f):
            table_new=table_new.drop(table_new.index[i])
            table_new=table_new.reset_index(drop=True)
            return table_new
    return table_new

In [14]:
def remove_outliers_pos2(table_old,table_new):
    for i in range(len(table_new)):
        k=table_new[np.where(table_new.index==i,True,False)]['ID2_G'].values[0]
        f=table_old[np.where(table_old.index==i,True,False)]['ID2_G'].values[0]
        if(k!=f):
            table_new=table_new.drop(table_new.index[i])
            table_new=table_new.reset_index(drop=True)
            return table_new
    return table_new

In [16]:
#Merge Position_atp with game_atp
df_rating_merge=df_rating_merge.fillna(-9999)
df_merged_first= df_merged_first.fillna(-9999)
df_rating_merge.columns=['ID1_G','POINT_R_1','POS_R_1','DATE_G_2','YEAR','MONTH','DAY']
temp=df_rating_merge
temp = temp.loc[:,['ID1_G','YEAR','DATE_G_2','POINT_R_1','POS_R_1','MONTH']]
df_merged_1 = pd.merge(df_merged_first, temp,on=['ID1_G','YEAR','DATE_G_2','MONTH'], how='left')
while(len(df_merged_1)!=len(df_merged_first)):
    df_merged_1=remove_outliers_pos(df_merged_first,df_merged_1)
df_rating_merge.columns=['ID2_G','POINT_R_1','POS_R_1','DATE_G_2','YEAR','MONTH','DAY']
temp.columns=['ID2_G','YEAR','DATE_G_2','POINT_R_2','POS_R_2','MONTH']
df_merged_2 = pd.merge( df_merged_first, temp,on=['ID2_G','YEAR','DATE_G_2','MONTH'], how='left')
while(len(df_merged_2)!=len(df_merged_first)):
    df_merged_2=remove_outliers_pos2(df_merged_first,df_merged_2)
df_merged_3=pd.merge(df_merged_1,df_merged_2,on=['ID1_G','ID2_G','YEAR','MONTH','DATE_G_2','ID_T_G','ID_R_G','RESULT_G','DAY'], how='left')

len(df_merged_3)

84485

In [ ]:
# Fonction permettant de donner une position aux joueurs pour des semaines "manquantes" dans la table rating

In [18]:
df_merged_3_pos_id1=df_merged_3
df_merged_3_pos_id1=df_merged_3_pos_id1.fillna(-9999)
debut=time.time()
for j in range(int(max(df_merged_3_pos_id1.ID2_G))):
    if(len(df_merged_3_pos_id1[df_merged_3_pos_id1.ID1_G==j])!=0):
        toto=df_merged_3_pos_id1[df_merged_3_pos_id1.ID1_G==j]
        index_true=toto.index
        if(sum(np.where(toto.POS_R_1!=-9999,1,0))!=0):
            for i in enumerate(index_true):
                if(i[0]!=0):
                    k=0
                    go=0
                    while(k<i[0] | go!=1 | k<5):
                        if(toto[np.where(toto.index==index_true[i[0]-1-k],True,False)]['POS_R_1'].values[0]!=-9999 and toto[np.where(toto.index==index_true[i[0]],True,False)]['POS_R_1'].values[0]==-9999):
                            df_merged_3_pos_id1['POS_R_1']=np.where(df_merged_3_pos_id1.index==index_true[i[0]],df_merged_3_pos_id1[df_merged_3_pos_id1.index==index_true[i[0]-1-k]]['POS_R_1'],df_merged_3_pos_id1['POS_R_1'])
                            toto=df_merged_3_pos_id1[df_merged_3_pos_id1.ID1_G==j]
                            go=1
                        k=k+1    
    if(j==100):
        fin=time.time()
        print(fin-debut)
    if(j==1000):
        fin=time.time()
        print(fin-debut)
    if(j==10000):
        fin=time.time()
        print(fin-debut)
df_merged_3_pos_id1

1.888207197189331
18.490959405899048
78.98668837547302


,ID1_G,ID2_G,ID_T_G,ID_R_G,RESULT_G,DATE_G_2,YEAR,MONTH,DAY,POINT_R_1,POS_R_1,POINT_R_2,POS_R_2
0,17613,18854,8887,4,1,5,2012,1,31,-9999.0,-9999.0,-9999.0,-9999.0
1,25191,25192,8888,4,2,5,2012,1,31,-9999.0,-9999.0,-9999.0,-9999.0
2,918,10828,8957,4,1,5,2012,1,31,375.0,144.0,175.0,271.0
3,3316,2379,8957,4,1,5,2012,1,31,254.0,193.0,381.0,141.0
4,7869,6277,8957,4,1,5,2012,1,31,223.0,215.0,352.0,152.0
5,7136,2107,8957,4,2,5,2012,1,31,140.0,317.0,134.0,329.0
6,72,678,8887,4,2,5,2012,1,31,899.0,51.0,1140.0,33.0
7,13796,563,8887,4,2,5,2012,1,31,688.0,72.0,-9999.0,-9999.0
8,775,745,8888,4,2,5,2012,1,31,243.0,198.0,547.0,105.0
9,75,1266,8888,4,1,5,2012,1,31,1025.0,39.0,710.0,67.0


In [19]:
df_merged_3_pos_id2=df_merged_3_pos_id1
df_merged_3_pos_id2=df_merged_3_pos_id2.fillna(-9999)
debut=time.time()
for j in range(int(max(df_merged_3_pos_id2.ID2_G))):
    if(len(df_merged_3_pos_id2[df_merged_3_pos_id2.ID2_G==j])!=0):
        toto=df_merged_3_pos_id2[df_merged_3_pos_id2.ID2_G==j]
        index_true=toto.index
        if(sum(np.where(toto.POS_R_2!=-9999,1,0))!=0):
            for i in enumerate(index_true):
                if(i[0]!=0):
                    k=0
                    go=0
                    while(k<i[0] | go!=1 | k<5):
                        if(toto[np.where(toto.index==index_true[i[0]-1-k],True,False)]['POS_R_2'].values[0]!=-9999 and toto[np.where(toto.index==index_true[i[0]],True,False)]['POS_R_2'].values[0]==-9999):
                            df_merged_3_pos_id2['POS_R_2']=np.where(df_merged_3_pos_id2.index==index_true[i[0]],df_merged_3_pos_id1[df_merged_3_pos_id2.index==index_true[i[0]-1-k]]['POS_R_2'],df_merged_3_pos_id2['POS_R_2'])
                            toto=df_merged_3_pos_id2[df_merged_3_pos_id2.ID2_G==j]
                            go=1
                        k=k+1    
    if(j==100):
        fin=time.time()
        print(fin-debut)
    if(j==1000):
        fin=time.time()
        print(fin-debut)
    if(j==10000):
        fin=time.time()
        print(fin-debut)
df_merged_3_pos_id2

1.7570521831512451
19.0830659866333
80.84305834770203


,ID1_G,ID2_G,ID_T_G,ID_R_G,RESULT_G,DATE_G_2,YEAR,MONTH,DAY,POINT_R_1,POS_R_1,POINT_R_2,POS_R_2
0,17613,18854,8887,4,1,5,2012,1,31,-9999.0,-9999.0,-9999.0,-9999.0
1,25191,25192,8888,4,2,5,2012,1,31,-9999.0,-9999.0,-9999.0,-9999.0
2,918,10828,8957,4,1,5,2012,1,31,375.0,144.0,175.0,271.0
3,3316,2379,8957,4,1,5,2012,1,31,254.0,193.0,381.0,141.0
4,7869,6277,8957,4,1,5,2012,1,31,223.0,215.0,352.0,152.0
5,7136,2107,8957,4,2,5,2012,1,31,140.0,317.0,134.0,329.0
6,72,678,8887,4,2,5,2012,1,31,899.0,51.0,1140.0,33.0
7,13796,563,8887,4,2,5,2012,1,31,688.0,72.0,-9999.0,-9999.0
8,775,745,8888,4,2,5,2012,1,31,243.0,198.0,547.0,105.0
9,75,1266,8888,4,1,5,2012,1,31,1025.0,39.0,710.0,67.0


In [ ]:
# Fonction permettant de donner les points aux joueurs pour des semaines "manquantes" dans la table rating

In [20]:
df_merged_3_poi_1=df_merged_3_pos_id2
df_merged_3_poi_1=df_merged_3_poi_1.fillna(-9999)
debut=time.time()
for j in range(int(max(df_merged_3_poi_1.ID2_G))):
    if(len(df_merged_3_poi_1[df_merged_3_poi_1.ID1_G==j])!=0):
        toto=df_merged_3_poi_1[df_merged_3_poi_1.ID1_G==j]
        index_true=toto.index
        if(sum(np.where(toto.POINT_R_1!=-9999,1,0))!=0):
            for i in enumerate(index_true):
                if(i[0]!=0):
                    k=0
                    go=0
                    while(k<i[0] | go!=1 | k<5):
                        if(toto[np.where(toto.index==index_true[i[0]-1-k],True,False)]['POINT_R_1'].values[0]!=-9999 and toto[np.where(toto.index==index_true[i[0]],True,False)]['POINT_R_1'].values[0]==-9999):
                            df_merged_3_poi_1['POINT_R_1']=np.where(df_merged_3_poi_1.index==index_true[i[0]],df_merged_3_poi_1[df_merged_3_poi_1.index==index_true[i[0]-1-k]]['POINT_R_1'],df_merged_3_poi_1['POINT_R_1'])
                            toto=df_merged_3_poi_1[df_merged_3_poi_1.ID1_G==j]
                            go=1
                        k=k+1    
    if(j==100):
        fin=time.time()
        print(fin-debut)
    if(j==1000):
        fin=time.time()
        print(fin-debut)
    if(j==10000):
        fin=time.time()
        print(fin-debut)
df_merged_3_poi_1

1.80792236328125
17.684327840805054
76.59311270713806


,ID1_G,ID2_G,ID_T_G,ID_R_G,RESULT_G,DATE_G_2,YEAR,MONTH,DAY,POINT_R_1,POS_R_1,POINT_R_2,POS_R_2
0,17613,18854,8887,4,1,5,2012,1,31,-9999.0,-9999.0,-9999.0,-9999.0
1,25191,25192,8888,4,2,5,2012,1,31,-9999.0,-9999.0,-9999.0,-9999.0
2,918,10828,8957,4,1,5,2012,1,31,375.0,144.0,175.0,271.0
3,3316,2379,8957,4,1,5,2012,1,31,254.0,193.0,381.0,141.0
4,7869,6277,8957,4,1,5,2012,1,31,223.0,215.0,352.0,152.0
5,7136,2107,8957,4,2,5,2012,1,31,140.0,317.0,134.0,329.0
6,72,678,8887,4,2,5,2012,1,31,899.0,51.0,1140.0,33.0
7,13796,563,8887,4,2,5,2012,1,31,688.0,72.0,-9999.0,-9999.0
8,775,745,8888,4,2,5,2012,1,31,243.0,198.0,547.0,105.0
9,75,1266,8888,4,1,5,2012,1,31,1025.0,39.0,710.0,67.0


In [21]:
df_merged_3_poi_2=df_merged_3_poi_1
df_merged_3_poi_2=df_merged_3_poi_2.fillna(-9999)
debut=time.time()
for j in range(int(max(df_merged_3_poi_2.ID2_G))):
    if(len(df_merged_3_poi_2[df_merged_3_poi_2.ID2_G==j])!=0):
        toto=df_merged_3_poi_2[df_merged_3_poi_2.ID2_G==j]
        index_true=toto.index
        if(sum(np.where(toto.POINT_R_1!=-9999,1,0))!=0):
            for i in enumerate(index_true):
                if(i[0]!=0):
                    k=0
                    go=0
                    while(k<i[0] | go!=1 | k<5):
                        if(toto[np.where(toto.index==index_true[i[0]-1-k],True,False)]['POINT_R_1'].values[0]!=-9999 and toto[np.where(toto.index==index_true[i[0]],True,False)]['POINT_R_2'].values[0]==-9999):
                            df_merged_3_poi_2['POINT_R_2']=np.where(df_merged_3_poi_2.index==index_true[i[0]],df_merged_3_poi_2[df_merged_3_poi_2.index==index_true[i[0]-1-k]]['POINT_R_2'],df_merged_3_poi_2['POINT_R_2'])
                            toto=df_merged_3_poi_2[df_merged_3_poi_2.ID2_G==j]
                            go=1
                        k=k+1    
    if(j==100):
        fin=time.time()
        print(fin-debut)
    if(j==1000):
        fin=time.time()
        print(fin-debut)
    if(j==10000):
        fin=time.time()
        print(fin-debut)
df_merged_3_poi_2

1.798095941543579
18.340317487716675
84.52931141853333


,ID1_G,ID2_G,ID_T_G,ID_R_G,RESULT_G,DATE_G_2,YEAR,MONTH,DAY,POINT_R_1,POS_R_1,POINT_R_2,POS_R_2
0,17613,18854,8887,4,1,5,2012,1,31,-9999.0,-9999.0,-9999.0,-9999.0
1,25191,25192,8888,4,2,5,2012,1,31,-9999.0,-9999.0,-9999.0,-9999.0
2,918,10828,8957,4,1,5,2012,1,31,375.0,144.0,175.0,271.0
3,3316,2379,8957,4,1,5,2012,1,31,254.0,193.0,381.0,141.0
4,7869,6277,8957,4,1,5,2012,1,31,223.0,215.0,352.0,152.0
5,7136,2107,8957,4,2,5,2012,1,31,140.0,317.0,134.0,329.0
6,72,678,8887,4,2,5,2012,1,31,899.0,51.0,1140.0,33.0
7,13796,563,8887,4,2,5,2012,1,31,688.0,72.0,-9999.0,-9999.0
8,775,745,8888,4,2,5,2012,1,31,243.0,198.0,547.0,105.0
9,75,1266,8888,4,1,5,2012,1,31,1025.0,39.0,710.0,67.0


In [22]:
print(len(df_merged_3_poi_2))

84485


In [23]:
df_merged_3_poi_2

,ID1_G,ID2_G,ID_T_G,ID_R_G,RESULT_G,DATE_G_2,YEAR,MONTH,DAY,POINT_R_1,POS_R_1,POINT_R_2,POS_R_2
0,17613,18854,8887,4,1,5,2012,1,31,-9999.0,-9999.0,-9999.0,-9999.0
1,25191,25192,8888,4,2,5,2012,1,31,-9999.0,-9999.0,-9999.0,-9999.0
2,918,10828,8957,4,1,5,2012,1,31,375.0,144.0,175.0,271.0
3,3316,2379,8957,4,1,5,2012,1,31,254.0,193.0,381.0,141.0
4,7869,6277,8957,4,1,5,2012,1,31,223.0,215.0,352.0,152.0
5,7136,2107,8957,4,2,5,2012,1,31,140.0,317.0,134.0,329.0
6,72,678,8887,4,2,5,2012,1,31,899.0,51.0,1140.0,33.0
7,13796,563,8887,4,2,5,2012,1,31,688.0,72.0,-9999.0,-9999.0
8,775,745,8888,4,2,5,2012,1,31,243.0,198.0,547.0,105.0
9,75,1266,8888,4,1,5,2012,1,31,1025.0,39.0,710.0,67.0


#### Test Pass : Position & Game_ATP
    

In [24]:

# drop all feature that isn't necessary
#df_merged_3=df_merged_3.drop(['POINT_R_1'],axis=1)
#df_merged_3=df_merged_3.drop(['POINT_R_2'],axis=1)
#df_merged_3=df_merged_3.drop(['TOP50'],axis=1)

dict_table['games_atp_public']
dict_table['games_atp_public']['DATE_G_2']= pd.to_datetime(dict_table['games_atp_public']['DATE_G'])
dict_table['games_atp_public']['DATE_G_2']=dict_table['games_atp_public']['DATE_G_2'].dt.week


test = dict_table["games_atp_public"]
test = test.apply(parsedate_atp,axis=1)
#test=test.drop(["DATE_G"],axis=1)
X_test_csv=test

#df_rating = dict_table['ratings_atp'][dict_table['ratings_atp'].DATE_G == '2016-07-25 00:00:00']

#test_rating=df_rating
#test_rating = test_rating.apply(parsedate_atp,axis=1)
#test_rating=test_rating.drop(["DATE_G"],axis=1)
X_test_csv
dict_table['ratings_atp']

,DATE_R,ID_P_R,POINT_R,POS_R,DATE_R_2
0,2012-01-02 00:00:00,5992,13630,1,1
1,2012-01-02 00:00:00,677,9595,2,1
2,2012-01-02 00:00:00,19,8170,3,1
3,2012-01-02 00:00:00,1075,7380,4,1
4,2012-01-02 00:00:00,673,4925,5,1
5,2012-01-02 00:00:00,3985,4335,6,1
6,2012-01-02 00:00:00,831,3700,7,1
7,2012-01-02 00:00:00,553,2965,8,1
8,2012-01-02 00:00:00,728,2595,9,1
9,2012-01-02 00:00:00,4045,2380,10,1


In [25]:
def parsedate_atp3(row):
    split_one=row['DATE_R'].split(' ')
    datetime=split_one[0].split('-')
    row['YEAR']=datetime[0]
    row['MONTH']=datetime[1]
    row['DAY']=datetime[2]
    return row

In [26]:
dict_table['ratings_atp']=dict_table['ratings_atp'].apply(parsedate_atp3,axis=1)


In [27]:
debut=time.time()
print(dict_table['ratings_atp'].columns)
dict_table['ratings_atp']['YEAR2']=-9999
dict_table['ratings_atp']['YEAR']=dict_table['ratings_atp']['YEAR'].astype('int')
for i in enumerate(dict_table['ratings_atp']['YEAR'].unique()):
    dict_table['ratings_atp']['YEAR2']=np.where(dict_table['ratings_atp']['YEAR']==i[1],i[1]-2011, dict_table['ratings_atp']['YEAR2'])
#dict_table['ratings_atp']['COMBO']=dict_table['ratings_atp']['DATE_R_2']*dict_table['ratings_atp']['']
dict_table['ratings_atp']['DATE_REAL']=-9999
dict_table['ratings_atp']['DATE_R_2']=dict_table['ratings_atp']['DATE_R_2'].astype('int')
for i in enumerate(dict_table['ratings_atp']['DATE_R_2'].unique()):
    dict_table['ratings_atp']['DATE_REAL']=np.where(dict_table['ratings_atp']['DATE_R_2']==i[1],(i[1]*dict_table['ratings_atp']['YEAR2']+53*(dict_table['ratings_atp']['YEAR2']-1)), dict_table['ratings_atp']['DATE_REAL'])
dict_table['ratings_atp']   

X_test_csv['POS_R_1']=-9999
for i in range(int(max(X_test_csv['ID2_G']))):
    if(len(X_test_csv.ID1_G)!=0):
        toto=dict_table['ratings_atp'][np.where(dict_table['ratings_atp'].ID_P_R==i,True,False)]
        k=362
        go=0
        while(go!=1 | k>350):
            if(len(np.where(toto[toto.DATE_REAL==k],1,0))!=0):
                X_test_csv['POS_R_1']=np.where(X_test_csv.ID1_G==i,toto[toto.DATE_REAL==k]['POS_R'].values[0],X_test_csv['POS_R_1'])
                go=1
            k=k-1
    if(i==1000):
        print("coucou")
        end=time.time()
        print(end-debut)
end=time.time()
print(end-debut)
X_test_csv

Index(['DATE_R', 'ID_P_R', 'POINT_R', 'POS_R', 'DATE_R_2', 'YEAR', 'MONTH',
       'DAY'],
      dtype='object')
coucou
6.101161956787109
344.11070585250854


,ID1_G,ID2_G,ID_T_G,ID_R_G,DATE_G,DATE_G_2,YEAR,MONTH,DAY,POS_R_1
0,30589,54796,13296,4,2016-08-01 00:00:00,31,2016,08,01,-9999
1,54409,24429,13296,4,2016-08-01 00:00:00,31,2016,08,01,-9999
2,28762,4025,13296,4,2016-08-01 00:00:00,31,2016,08,01,428
3,26009,19584,13296,4,2016-08-01 00:00:00,31,2016,08,01,617
4,12430,12767,13296,4,2016-08-01 00:00:00,31,2016,08,01,667
5,54797,24310,13296,4,2016-08-01 00:00:00,31,2016,08,01,-9999
6,54790,26730,13296,4,2016-08-01 00:00:00,31,2016,08,01,-9999
7,54792,54791,13296,4,2016-08-01 00:00:00,31,2016,08,01,-9999
8,11948,24531,13298,3,2016-08-01 00:00:00,31,2016,08,01,470
9,17928,36821,13298,3,2016-08-01 00:00:00,31,2016,08,01,138


In [28]:
X_test_csv['POS_R_2']=-9999
for i in range(int(max(X_test_csv['ID2_G']))):
    if(len(X_test_csv.ID2_G)!=0):
        toto=dict_table['ratings_atp'][np.where(dict_table['ratings_atp'].ID_P_R==i,True,False)]
        k=362
        go=0
        while(go!=1 | k>350):
            if(len(np.where(toto[toto.DATE_REAL==k],1,0))!=0):
                X_test_csv['POS_R_2']=np.where(X_test_csv.ID2_G==i,toto[toto.DATE_REAL==k]['POS_R'].values[0],X_test_csv['POS_R_2'])
                go=1
            k=k-1
    if(i==1000):
        print("coucou")
        end=time.time()
        print(end-debut)
end=time.time()
print(end-debut)
X_test_csv

coucou
350.3792004585266
702.888388633728


,ID1_G,ID2_G,ID_T_G,ID_R_G,DATE_G,DATE_G_2,YEAR,MONTH,DAY,POS_R_1,POS_R_2
0,30589,54796,13296,4,2016-08-01 00:00:00,31,2016,08,01,-9999,-9999
1,54409,24429,13296,4,2016-08-01 00:00:00,31,2016,08,01,-9999,-9999
2,28762,4025,13296,4,2016-08-01 00:00:00,31,2016,08,01,428,214
3,26009,19584,13296,4,2016-08-01 00:00:00,31,2016,08,01,617,288
4,12430,12767,13296,4,2016-08-01 00:00:00,31,2016,08,01,667,354
5,54797,24310,13296,4,2016-08-01 00:00:00,31,2016,08,01,-9999,-9999
6,54790,26730,13296,4,2016-08-01 00:00:00,31,2016,08,01,-9999,-9999
7,54792,54791,13296,4,2016-08-01 00:00:00,31,2016,08,01,-9999,-9999
8,11948,24531,13298,3,2016-08-01 00:00:00,31,2016,08,01,470,880
9,17928,36821,13298,3,2016-08-01 00:00:00,31,2016,08,01,138,791


In [66]:
X_test_csv['POINT_R_2']=-9999
for i in range(int(max(X_test_csv['ID2_G']))):
    if(len(X_test_csv.ID2_G)!=0):
        toto=dict_table['ratings_atp'][np.where(dict_table['ratings_atp'].ID_P_R==i,True,False)]
        k=362
        go=0
        while(go!=1 | k>350):
            if(len(np.where(toto[toto.DATE_REAL==k],1,0))!=0):
                X_test_csv['POINT_R_2']=np.where(X_test_csv.ID2_G==i,toto[toto.DATE_REAL==k]['POINT_R'].values[0],X_test_csv['POINT_R_2'])
                go=1
            k=k-1
    if(i==1000):
        print("coucou")
        end=time.time()
        print(end-debut)
end=time.time()
print(end-debut)
X_test_csv

coucou
957.5492539405823
1301.0127153396606


,ID1_G,ID2_G,ID_T_G,ID_R_G,YEAR,MONTH,DAY,POS_R_1,POS_R_2,POINT_R_1,POINT_R_2
0,30589,54796,13296,4,2016,08,01,-9999,-9999,-9999,-9999
1,54409,24429,13296,4,2016,08,01,-9999,-9999,-9999,-9999
2,28762,4025,13296,4,2016,08,01,428,214,93,237
3,26009,19584,13296,4,2016,08,01,617,288,50,173
4,12430,12767,13296,4,2016,08,01,667,354,41,130
5,54797,24310,13296,4,2016,08,01,-9999,-9999,-9999,-9999
6,54790,26730,13296,4,2016,08,01,-9999,-9999,-9999,-9999
7,54792,54791,13296,4,2016,08,01,-9999,-9999,-9999,-9999
8,11948,24531,13298,3,2016,08,01,470,880,85,20
9,17928,36821,13298,3,2016,08,01,138,791,431,26


In [29]:
X_test_csv['POINT_R_1']=-9999
for i in range(int(max(X_test_csv['ID2_G']))):
    if(len(X_test_csv.ID1_G)!=0):
        toto=dict_table['ratings_atp'][np.where(dict_table['ratings_atp'].ID_P_R==i,True,False)]
        k=362
        go=0
        while(go!=1 | k>350):
            if(len(np.where(toto[toto.DATE_REAL==k],1,0))!=0):
                X_test_csv['POINT_R_1']=np.where(X_test_csv.ID1_G==i,toto[toto.DATE_REAL==k]['POINT_R'].values[0],X_test_csv['POINT_R_1'])
                go=1
            k=k-1
    if(i==1000):
        print("coucou")
        end=time.time()
        print(end-debut)
end=time.time()
print(end-debut)
X_test_csv

coucou
708.9799914360046
1046.4070596694946


,ID1_G,ID2_G,ID_T_G,ID_R_G,DATE_G,DATE_G_2,YEAR,MONTH,DAY,POS_R_1,POS_R_2,POINT_R_1
0,30589,54796,13296,4,2016-08-01 00:00:00,31,2016,08,01,-9999,-9999,-9999
1,54409,24429,13296,4,2016-08-01 00:00:00,31,2016,08,01,-9999,-9999,-9999
2,28762,4025,13296,4,2016-08-01 00:00:00,31,2016,08,01,428,214,93
3,26009,19584,13296,4,2016-08-01 00:00:00,31,2016,08,01,617,288,50
4,12430,12767,13296,4,2016-08-01 00:00:00,31,2016,08,01,667,354,41
5,54797,24310,13296,4,2016-08-01 00:00:00,31,2016,08,01,-9999,-9999,-9999
6,54790,26730,13296,4,2016-08-01 00:00:00,31,2016,08,01,-9999,-9999,-9999
7,54792,54791,13296,4,2016-08-01 00:00:00,31,2016,08,01,-9999,-9999,-9999
8,11948,24531,13298,3,2016-08-01 00:00:00,31,2016,08,01,470,880,85
9,17928,36821,13298,3,2016-08-01 00:00:00,31,2016,08,01,138,791,431


In [ ]:
X_test_csv[X_test_csv.ID2_G==5992]

In [ ]:
dict_table['games_atp_public'][dict_table['games_atp_public'].ID2_G==5992]

### Pass X_train format : 

In [69]:
#df_merged_3=df_merged_3.drop('TOP50',axis=1)
df_merged_3_poi_2=df_merged_3_poi_2.fillna(-9999)
X_train=df_merged_3_poi_2
Y_train = X_train['RESULT_G']
X_train= X_train.drop(["RESULT_G"],axis=1)
#X_train=X_train.drop(['DATE_G_2'],axis=1)
#X_test_csv=X_test_csv.drop(['DATE_G_2'],axis=1)
#X_test_csv=X_test_csv.drop(['DATE_G'],axis=1)
X_test=X_test_csv
X_train

,ID1_G,ID2_G,ID_T_G,ID_R_G,DATE_G_2,YEAR,MONTH,DAY,POINT_R_1,POS_R_1,POINT_R_2,POS_R_2
0,17613,18854,8887,4,5,2012,1,31,-9999.0,-9999.0,-9999.0,-9999.0
1,25191,25192,8888,4,5,2012,1,31,-9999.0,-9999.0,-9999.0,-9999.0
2,918,10828,8957,4,5,2012,1,31,375.0,144.0,175.0,271.0
3,3316,2379,8957,4,5,2012,1,31,254.0,193.0,381.0,141.0
4,7869,6277,8957,4,5,2012,1,31,223.0,215.0,352.0,152.0
5,7136,2107,8957,4,5,2012,1,31,140.0,317.0,134.0,329.0
6,72,678,8887,4,5,2012,1,31,899.0,51.0,1140.0,33.0
7,13796,563,8887,4,5,2012,1,31,688.0,72.0,-9999.0,-9999.0
8,775,745,8888,4,5,2012,1,31,243.0,198.0,547.0,105.0
9,75,1266,8888,4,5,2012,1,31,1025.0,39.0,710.0,67.0


In [70]:
#Prepare data split 70/30 to hidden database
def prepare_database_hidden(X_train,Y_train):
    X_train=X_train.fillna(-9999)
    Y_train=Y_train.fillna(-9999)
    X_test=Y_train.fillna(-9999)
    tmp=len(X_train.YEAR[X_train.YEAR!=2016])
    X_test_h=X_train[tmp+1:len(X_train)]
    X_train_h=X_train[0:tmp]

    Y_test_h=Y_train[tmp+1:len(Y_train)]
    Y_train_h=Y_train[0:tmp]
    
    return X_train_h,Y_train_h,X_test_h,Y_test_h


In [71]:
X_train_h,Y_train_h,X_test_h,Y_test_h=prepare_database_hidden(X_train,Y_train)
print(len(Y_train_h)," ",Y_train_h.shape)
print(len(X_test_h)," ",X_test_h.shape)
print(len(Y_test_h)," ",Y_test_h.shape)
print(len(X_train_h)," ",X_train_h.shape)


Create_data_toevaluate(X_train_h,"X_train_1_pos_12f_sem_hidden",Y_train_h,"Y_train_1_pos_12f_sem_hidden",X_test_h,"X_test_1_pos_12f_sem_hidden",Y_test_h,"Y_test_1_pos_12f_sem_hidden")

62865   (62865,)
21619   (21619, 12)
21619   (21619,)
62865   (62865, 12)


### Create X_train,Y_train,X_test_leaderboard with 12 FEATURES

In [ ]:
Create_data_toevaluate(X_train,"X_train_1_pos_12f_sem",Y_train,"Y_train_1_pos_12f_sem",X_test,"X_test_1_pos_12f_sem",Y_test," ")

## New Feature % Game Win

In [72]:
def game_win_all(table):
    for i in range(max(table.ID_P)):
        if(len(dict_table['games_atp'][dict_table['games_atp'].ID1_G==i])!=0 and len(dict_table['games_atp'][dict_table['games_atp'].ID2_G==i])!=0):
            id1=len(dict_table['games_atp'][dict_table['games_atp'].ID1_G==i])
            id2=len(dict_table['games_atp'][dict_table['games_atp'].ID2_G==i])
             #sum(np.where(dict_table['games_atp']['ID1_G']==5992, 1,0))
            id_final=id1/(id1+id2)
        else:
            id_final=0
        table.at[table.ID_P==i,'WIN_POUR']=id_final
    return table

In [73]:
import time
debut=time.time()
dict_table['players_atp']=game_win_all(dict_table['players_atp'])
fin=time.time()
print(fin-debut)

148.5515124797821


In [262]:
temp=dict_table['players_atp']
temp = temp.loc[:,['ID_P', 'WIN_POUR']]
temp.columns=['ID1_G','WIN_POUR_1']

In [263]:
#Merge Position_atp with game_atp with players
df_merged_3_poi_2=df_merged_3_poi_2.fillna(-9999)
df_merged_1 = pd.merge(df_merged_3_poi_2,temp,left_on=['ID1_G'], right_on=['ID1_G'], how='left')
temp.columns=['ID2_G','WIN_POUR_2']
df_merged_2 =pd.merge(df_merged_3_poi_2,temp,left_on=['ID2_G'],  right_on=['ID2_G'], how='left')
df_merged_1=df_merged_1.loc[:,['ID1_G','ID2_G','YEAR','MONTH','DAY', 'WIN_POUR_1']]
df_merged_3_play_pos=pd.merge(df_merged_1,df_merged_2,left_on=['ID1_G','ID2_G','YEAR','MONTH','DAY'],right_on=['ID1_G','ID2_G','YEAR','MONTH','DAY'], how='inner')
df_merged_3_play_pos=df_merged_3_play_pos.fillna(-9999)
df_merged_3_play_pos

,ID1_G,ID2_G,YEAR,MONTH,DAY,WIN_POUR_1,ID_T_G,ID_R_G,RESULT_G,DATE_G_2,POINT_R_1,POS_R_1,POINT_R_2,POS_R_2,WIN_POUR_2
0,17613,18854,2012,1,31,0.461538,8887,4,1,5,-9999.0,-9999.0,-9999.0,-9999.0,0.478261
1,25191,25192,2012,1,31,0.000000,8888,4,2,5,-9999.0,-9999.0,-9999.0,-9999.0,0.666667
2,918,10828,2012,1,31,0.558282,8957,4,1,5,375.0,144.0,175.0,271.0,0.576052
3,3316,2379,2012,1,31,0.548822,8957,4,1,5,254.0,193.0,381.0,141.0,0.529183
4,7869,6277,2012,1,31,0.378378,8957,4,1,5,223.0,215.0,352.0,152.0,0.419753
5,7136,2107,2012,1,31,0.315789,8957,4,2,5,140.0,317.0,134.0,329.0,0.558559
6,72,678,2012,1,31,0.487395,8887,4,2,5,899.0,51.0,1140.0,33.0,0.595890
7,13796,563,2012,1,31,0.514286,8887,4,2,5,688.0,72.0,-9999.0,-9999.0,0.550877
8,775,745,2012,1,31,0.532258,8888,4,2,5,243.0,198.0,547.0,105.0,0.570934
9,75,1266,2012,1,31,0.556863,8888,4,1,5,1025.0,39.0,710.0,67.0,0.547619


In [76]:
def split_X_train_val(X):
    X_train=X
    Y_train = X_train['RESULT_G']
    X_train= X_train.drop(["RESULT_G"],axis=1)
    X_train=X_train.fillna(-9999)
    Y_train=Y_train.fillna(-9999)
    tmp=len(X_train.YEAR[df_merged_3_play_pos.YEAR!='2016'])
    X_test=X_train[tmp:len(X_train)]
    X_train=X_train[0:tmp-1]
    Y_test=Y_train[tmp:len(Y_train)]
    Y_train=Y_train[0:tmp-1]

    print(len(Y_train))
    print(len(X_test))
    print(len(Y_test))
    print(len(X_train))
    return X_train,Y_train,X_test,Y_test

#### PARTIE TEST Win %

In [77]:
def no_split_X_train(X):
    X_train=X
    if('RESULT_G' in X_train.columns):
        Y_train = X_train['RESULT_G']
        X_train= X_train.drop(["RESULT_G"],axis=1)
        Y_train=Y_train.fillna(-9999)
    else:
        Y_train=0
    X_train=X_train.fillna(-9999)

    return X_train,Y_train

In [264]:
#Merge Position_atp with game_atp with players
temp.columns=['ID1_G','WIN_POUR_1']
df_merged_1 = pd.merge( X_test_csv,temp,left_on=['ID1_G'], right_on=['ID1_G'], how='left')
temp.columns=['ID2_G','WIN_POUR_2']
df_merged_2 =pd.merge( X_test_csv,temp,left_on=['ID2_G'],  right_on=['ID2_G'], how='left')
df_merged_1=df_merged_1.loc[:,['ID1_G','ID2_G','YEAR','MONTH','DAY', 'WIN_POUR_1']]
df_merged_3_play_pos_test=pd.merge(df_merged_1,df_merged_2,left_on=['ID1_G','ID2_G','YEAR','MONTH','DAY'],right_on=['ID1_G','ID2_G','YEAR','MONTH','DAY'], how='inner')
df_merged_3_play_pos_test=df_merged_3_play_pos_test.fillna(-9999)

In [267]:
print(len(df_merged_3_play_pos_test))
print(df_merged_3_play_pos.columns)

7608
Index(['ID1_G', 'ID2_G', 'YEAR', 'MONTH', 'DAY', 'WIN_POUR_1', 'ID_T_G',
       'ID_R_G', 'RESULT_G', 'DATE_G_2', 'POINT_R_1', 'POS_R_1', 'POINT_R_2',
       'POS_R_2', 'WIN_POUR_2'],
      dtype='object')


In [268]:
df_merged_3_play_pos

,ID1_G,ID2_G,YEAR,MONTH,DAY,WIN_POUR_1,ID_T_G,ID_R_G,RESULT_G,DATE_G_2,POINT_R_1,POS_R_1,POINT_R_2,POS_R_2,WIN_POUR_2
0,17613,18854,2012,1,31,0.461538,8887,4,1,5,-9999.0,-9999.0,-9999.0,-9999.0,0.478261
1,25191,25192,2012,1,31,0.000000,8888,4,2,5,-9999.0,-9999.0,-9999.0,-9999.0,0.666667
2,918,10828,2012,1,31,0.558282,8957,4,1,5,375.0,144.0,175.0,271.0,0.576052
3,3316,2379,2012,1,31,0.548822,8957,4,1,5,254.0,193.0,381.0,141.0,0.529183
4,7869,6277,2012,1,31,0.378378,8957,4,1,5,223.0,215.0,352.0,152.0,0.419753
5,7136,2107,2012,1,31,0.315789,8957,4,2,5,140.0,317.0,134.0,329.0,0.558559
6,72,678,2012,1,31,0.487395,8887,4,2,5,899.0,51.0,1140.0,33.0,0.595890
7,13796,563,2012,1,31,0.514286,8887,4,2,5,688.0,72.0,-9999.0,-9999.0,0.550877
8,775,745,2012,1,31,0.532258,8888,4,2,5,243.0,198.0,547.0,105.0,0.570934
9,75,1266,2012,1,31,0.556863,8888,4,1,5,1025.0,39.0,710.0,67.0,0.547619


In [269]:
#df_merged_3_play_pos=df_merged_3_play_pos.drop('TOP50',axis=1)
X_test=df_merged_3_play_pos_test
X_train,Y_train=no_split_X_train(df_merged_3_play_pos)
X_train.shape
X_train=X_train.drop('DATE_G_2',axis=1)
df_merged_3_play_pos_test.shape

(7608, 13)

In [270]:
X_train_h,Y_train_h,X_test_h,Y_test_h=prepare_database_hidden(X_train,Y_train)
print(len(Y_train_h)," ",Y_train_h.shape)
print(len(X_test_h)," ",X_test_h.shape)
print(len(Y_test_h)," ",Y_test_h.shape)
print(len(X_train_h)," ",X_train_h.shape)

62865   (62865,)
21619   (21619, 13)
21619   (21619,)
62865   (62865, 13)


In [271]:
Create_data_toevaluate(X_train_h,"X_train_2_pos_win_14f_sem_hidden",Y_train_h,"Y_train_2_pos_win_14f_sem_hidden",X_test_h,"X_test_2_pos_win_14f_sem_hidden",Y_test_h,"Y_test_2_pos_win_14f_sem_hidden")

In [272]:
Create_data_toevaluate(X_train,"X_train_2_pos_win_14f_sem",Y_train,"Y_train_2_pos_win_14f_sem",X_test,"X_test_2_pos_win_14f_sem",Y_test," ")

## NEW FEATURE ID TOURNEY

#### Pass Train : ID_TOURNEY

In [242]:
dict_table['tours_atp']['ID_T_REAL']=0

Fonction permettant de mettre un indice unique au différents tournois

In [243]:
def new_tourney_id(table):
    for indice,valeur in enumerate(table['NAME_T'].unique()):
        table['ID_T_REAL']=np.where(table['NAME_T']==valeur,indice,table['ID_T_REAL'])
    return table

In [244]:
dict_table['tours_atp']=new_tourney_id(dict_table['tours_atp'])

In [245]:
temp=dict_table['tours_atp']
temp = temp.loc[:,['ID_T', 'ID_T_REAL','ID_C_T']]

In [246]:
#Merge Position_atp with game_atp with players
temp.columns=['ID_T_G','ID_T_REAL','ID_C_T']
df_merged_pos_tourney = pd.merge( df_merged_3_play_pos,temp,left_on=['ID_T_G'], right_on=['ID_T_G'], how='left')
df_merged_pos_tourney.fillna(-9999)
df_merged_pos_tourney

,ID1_G,ID2_G,YEAR,MONTH,DAY,WIN_POUR_1,ID_T_G,ID_R_G,RESULT_G,DATE_G_2,POINT_R_1,POS_R_1,POINT_R_2,POS_R_2,WIN_POUR_2,ID_T_REAL,ID_C_T
0,17613,18854,2012,1,31,0.461538,8887,4,1,5,-9999.0,-9999.0,-9999.0,-9999.0,0.478261,3300,3
1,25191,25192,2012,1,31,0.000000,8888,4,2,5,-9999.0,-9999.0,-9999.0,-9999.0,0.666667,2460,3
2,918,10828,2012,1,31,0.558282,8957,4,1,5,375.0,144.0,175.0,271.0,0.576052,3317,3
3,3316,2379,2012,1,31,0.548822,8957,4,1,5,254.0,193.0,381.0,141.0,0.529183,3317,3
4,7869,6277,2012,1,31,0.378378,8957,4,1,5,223.0,215.0,352.0,152.0,0.419753,3317,3
5,7136,2107,2012,1,31,0.315789,8957,4,2,5,140.0,317.0,134.0,329.0,0.558559,3317,3
6,72,678,2012,1,31,0.487395,8887,4,2,5,899.0,51.0,1140.0,33.0,0.595890,3300,3
7,13796,563,2012,1,31,0.514286,8887,4,2,5,688.0,72.0,-9999.0,-9999.0,0.550877,3300,3
8,775,745,2012,1,31,0.532258,8888,4,2,5,243.0,198.0,547.0,105.0,0.570934,2460,3
9,75,1266,2012,1,31,0.556863,8888,4,1,5,1025.0,39.0,710.0,67.0,0.547619,2460,3


#### Pass test : ID Tourney

In [247]:
temp=dict_table['tours_atp']
temp = temp.loc[:,['ID_T', 'ID_T_REAL','ID_C_T']]
#Merge Position_atp with game_atp with players
temp.columns=['ID_T_G','ID_T_REAL','ID_C_T']
df_merged_pos_tourney_test = pd.merge(df_merged_3_play_pos_test,temp,left_on=['ID_T_G'], right_on=['ID_T_G'], how='left')
df_merged_pos_tourney_test=df_merged_pos_tourney_test.fillna(-9999)
df_merged_pos_tourney_test

,ID1_G,ID2_G,YEAR,MONTH,DAY,WIN_POUR_1,ID_T_G,ID_R_G,POS_R_1,POS_R_2,POINT_R_1,POINT_R_2,WIN_POUR_2,ID_T_REAL,ID_C_T
0,30589,54796,2016,08,01,0.545455,13296,4,-9999,-9999,-9999,-9999,0.000000,4360,1
1,54409,24429,2016,08,01,0.500000,13296,4,-9999,-9999,-9999,-9999,0.628571,4360,1
2,28762,4025,2016,08,01,0.636364,13296,4,428,214,93,237,0.440191,4360,1
3,26009,19584,2016,08,01,0.441176,13296,4,617,288,50,173,0.475610,4360,1
4,12430,12767,2016,08,01,0.553191,13296,4,667,354,41,130,0.436170,4360,1
5,54797,24310,2016,08,01,0.000000,13296,4,-9999,-9999,-9999,-9999,0.423077,4360,1
6,54790,26730,2016,08,01,0.000000,13296,4,-9999,-9999,-9999,-9999,0.000000,4360,1
7,54792,54791,2016,08,01,0.000000,13296,4,-9999,-9999,-9999,-9999,0.000000,4360,1
8,11948,24531,2016,08,01,0.533333,13298,3,470,880,85,20,0.428571,3876,2
9,17928,36821,2016,08,01,0.546816,13298,3,138,791,431,26,0.555556,3876,2


## NEW FEATURE %WIN EACH TYPE OF COURT

#### Train Pass: %Win Type of court 

In [248]:
def tourney_court_win(table):
    for i in range(int(max(table.ID2_G))):
        if(len(table[table.ID1_G==i])!=0 and len(table[table.ID2_G==i])!=0):
            for k in range(1,6):
                tempbis=table[np.where(((table.ID1_G==i)|(table.ID2_G==i)) & (table.ID_C_T==k),True,False)]
                if(len(tempbis)!=0):
                    #id1=len(temp[temp.ID1_G==i])
                    id1=sum(np.where(tempbis['ID1_G']==i, 1,0))
                    #id2=len(temp[temp.ID2_G==i])
                    id2=sum(np.where(tempbis['ID2_G']==i, 1,0))
                    id_final=id1/(id1+id2)
                    table['WIN_COURT_1']=np.where((table.ID1_G==i) & (table.ID_C_T==k),id_final,table['WIN_COURT_1'])
                    table['WIN_COURT_2']=np.where((table.ID2_G==i) & (table.ID_C_T==k),id_final,table['WIN_COURT_2'])
    return table

In [249]:
df_merged_pos_tourney['WIN_COURT_1']=-9999
df_merged_pos_tourney['WIN_COURT_2']=-9999
df_merged_pos_tourney=tourney_court_win(df_merged_pos_tourney)
df_merged_pos_tourney=df_merged_pos_tourney.fillna(-9999)

In [250]:
df_merged_pos_tourney

,ID1_G,ID2_G,YEAR,MONTH,DAY,WIN_POUR_1,ID_T_G,ID_R_G,RESULT_G,DATE_G_2,POINT_R_1,POS_R_1,POINT_R_2,POS_R_2,WIN_POUR_2,ID_T_REAL,ID_C_T,WIN_COURT_1,WIN_COURT_2
0,17613,18854,2012,1,31,0.461538,8887,4,1,5,-9999.0,-9999.0,-9999.0,-9999.0,0.478261,3300,3,0.428571,0.611111
1,25191,25192,2012,1,31,0.000000,8888,4,2,5,-9999.0,-9999.0,-9999.0,-9999.0,0.666667,2460,3,-9999.000000,0.500000
2,918,10828,2012,1,31,0.558282,8957,4,1,5,375.0,144.0,175.0,271.0,0.576052,3317,3,0.474576,0.464286
3,3316,2379,2012,1,31,0.548822,8957,4,1,5,254.0,193.0,381.0,141.0,0.529183,3317,3,0.640000,0.285714
4,7869,6277,2012,1,31,0.378378,8957,4,1,5,223.0,215.0,352.0,152.0,0.419753,3317,3,0.400000,0.428571
5,7136,2107,2012,1,31,0.315789,8957,4,2,5,140.0,317.0,134.0,329.0,0.558559,3317,3,0.500000,0.357143
6,72,678,2012,1,31,0.487395,8887,4,2,5,899.0,51.0,1140.0,33.0,0.595890,3300,3,0.617647,0.482759
7,13796,563,2012,1,31,0.514286,8887,4,2,5,688.0,72.0,-9999.0,-9999.0,0.550877,3300,3,0.900000,0.468085
8,775,745,2012,1,31,0.532258,8888,4,2,5,243.0,198.0,547.0,105.0,0.570934,2460,3,0.425926,0.495652
9,75,1266,2012,1,31,0.556863,8888,4,1,5,1025.0,39.0,710.0,67.0,0.547619,2460,3,0.551020,0.510000


#### Test Pass : %Win each tourney

In [251]:
#df_merged_pos_tourney_test_J1
df_merged_column=df_merged_pos_tourney.loc[:,['ID1_G','ID_C_T','WIN_COURT_1']]
df_merged_column['ID1_G']=df_merged_column['ID1_G'].astype(int)
df_merged_test_final_test4_1=pd.merge(df_merged_pos_tourney_test,df_merged_column,on=['ID1_G','ID_C_T'],how='left')
df_merged_test_final_test4_1=df_merged_test_final_test4_1.drop_duplicates(keep='last')
print(len(df_merged_test_final_test4_1))

#df_merged_pos_tourney_test_J2
df_merged_column=df_merged_pos_tourney.loc[:,['ID2_G','ID_C_T','WIN_COURT_2']]
df_merged_column['ID2_G']=df_merged_column['ID2_G'].astype(int)

df_merged_test_final_test4_2=pd.merge(df_merged_pos_tourney_test,df_merged_column,on=['ID2_G','ID_C_T'],how='left')
df_merged_test_final_test4_2=df_merged_test_final_test4_2.drop_duplicates(keep='last')
print(len(df_merged_test_final_test4_2))

df_merged_test_final_test_court=pd.merge(df_merged_test_final_test4_1,df_merged_test_final_test4_2, on=['ID1_G','ID2_G','YEAR','MONTH','DAY','WIN_POUR_1','WIN_POUR_2','POS_R_1','POS_R_2','ID_R_G','ID_T_G','ID_T_REAL','ID_C_T','POINT_R_1','POINT_R_2'],how='left')
df_merged_test_final_test_court=df_merged_test_final_test_court.fillna(-9999)

7608
7608


In [252]:
df_merged_test_final_test_court

,ID1_G,ID2_G,YEAR,MONTH,DAY,WIN_POUR_1,ID_T_G,ID_R_G,POS_R_1,POS_R_2,POINT_R_1,POINT_R_2,WIN_POUR_2,ID_T_REAL,ID_C_T,WIN_COURT_1,WIN_COURT_2
0,30589,54796,2016,08,01,0.545455,13296,4,-9999,-9999,-9999,-9999,0.000000,4360,1,-9999.000000,-9999.000000
1,54409,24429,2016,08,01,0.500000,13296,4,-9999,-9999,-9999,-9999,0.628571,4360,1,-9999.000000,0.529412
2,28762,4025,2016,08,01,0.636364,13296,4,428,214,93,237,0.440191,4360,1,0.545455,0.538462
3,26009,19584,2016,08,01,0.441176,13296,4,617,288,50,173,0.475610,4360,1,0.490909,0.409836
4,12430,12767,2016,08,01,0.553191,13296,4,667,354,41,130,0.436170,4360,1,0.483871,0.521127
5,54797,24310,2016,08,01,0.000000,13296,4,-9999,-9999,-9999,-9999,0.423077,4360,1,-9999.000000,0.578947
6,54790,26730,2016,08,01,0.000000,13296,4,-9999,-9999,-9999,-9999,0.000000,4360,1,-9999.000000,-9999.000000
7,54792,54791,2016,08,01,0.000000,13296,4,-9999,-9999,-9999,-9999,0.000000,4360,1,-9999.000000,-9999.000000
8,11948,24531,2016,08,01,0.533333,13298,3,470,880,85,20,0.428571,3876,2,0.536585,0.640000
9,17928,36821,2016,08,01,0.546816,13298,3,138,791,431,26,0.555556,3876,2,0.482014,0.450000


In [253]:
X_train,Y_train=no_split_X_train(df_merged_pos_tourney)
X_test=df_merged_test_final_test_court
X_train=X_train.drop(['DATE_G_2'],axis=1)
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)

(84485, 17)
(84485,)
(7608, 17)


In [254]:
X_train_h,Y_train_h,X_test_h,Y_test_h=prepare_database_hidden(X_train,Y_train)
print(len(Y_train_h)," ",Y_train_h.shape)
print(len(X_test_h)," ",X_test_h.shape)
print(len(Y_test_h)," ",Y_test_h.shape)
print(len(X_train_h)," ",X_train_h.shape)

62865   (62865,)
21619   (21619, 17)
21619   (21619,)
62865   (62865, 17)


In [255]:
Create_data_toevaluate(X_train_h,"X_train_3_pos_win_17f_sem_hidden",Y_train_h,"Y_train_3_pos_win_17f_sem_hidden",X_test_h,"X_test_3_pos_win_17f_sem_hidden",Y_test_h,"Y_test_3_pos_win_17f_sem_hidden")

In [256]:
Create_data_toevaluate(X_train,"X_train_3_pos_win_17f_sem",Y_train,"Y_train_3_pos_win_17f_sem",X_test,"X_test_3_pos_win_17f_sem",Y_test," ")

## NEW FEATURE %WIN FOR EACH TOURNEY

Train pass : %Win for each Tourney

In [97]:
def game_win_tourney(table):
    for i in range(int(max(df_merged_pos_tourney2.ID2_G))):
        if(len(table[table.ID1_G==i])!=0 and len(table[table.ID2_G==i])!=0):
            tempbis=table[np.where((table.ID1_G==i) | (table.ID_T_REAL==i),True,False)]
            for j in enumerate(tempbis.index):
                toto=tempbis[np.where(tempbis.index==j[1],True,False)]['ID_T_REAL'].values[0]
                temp=table[table.ID_T_REAL==toto]
                if(len(temp[temp.ID2_G==i])!=0 and len(temp[temp.ID1_G==i])!=0):
                    #id1=len(temp[temp.ID1_G==i])
                    id1=sum(np.where(temp['ID1_G']==i, 1,0))
                    #id2=len(temp[temp.ID2_G==i])
                    id2=sum(np.where(temp['ID2_G']==i, 1,0))
                    id_final=id1/(id1+id2)
                else:
                    id_final=0
                table['WIN_FOR_TOURNEY_1']=np.where((table.ID1_G==i) & (table.ID_T_REAL==toto),id_final,table['WIN_FOR_TOURNEY_1'])
                table['WIN_FOR_TOURNEY_2']=np.where((table.ID2_G==i) & (table.ID_T_REAL==toto),id_final,table['WIN_FOR_TOURNEY_2'])
    return table

In [98]:
debut=time.time()
df_merged_pos_tourney2=df_merged_pos_tourney
df_merged_pos_tourney2['WIN_FOR_TOURNEY_1']=0
df_merged_pos_tourney2['WIN_FOR_TOURNEY_2']=0
df_merged_pos_tourney2=game_win_tourney(df_merged_pos_tourney2)
fin=time.time()
print("temps",fin-debut)

temps 324.56716799736023


In [99]:
df_merged_pos_tourney2=df_merged_pos_tourney2.fillna(-9999)


In [100]:
df_merged_pos_tourney2

,ID1_G,ID2_G,YEAR,MONTH,DAY,WIN_POUR_1,ID_T_G,ID_R_G,RESULT_G,DATE_G_2,...,POS_R_1,POINT_R_2,POS_R_2,WIN_POUR_2,ID_T_REAL,ID_C_T,WIN_COURT_1,WIN_COURT_2,WIN_FOR_TOURNEY_1,WIN_FOR_TOURNEY_2
0,17613,18854,2012,1,31,0.461538,8887,4,1,5,...,-9999.0,-9999.0,-9999.0,0.478261,3300,3,0.428571,0.611111,0.500000,0.000000
1,25191,25192,2012,1,31,0.000000,8888,4,2,5,...,-9999.0,-9999.0,-9999.0,0.666667,2460,3,-9999.000000,0.500000,0.000000,0.500000
2,918,10828,2012,1,31,0.558282,8957,4,1,5,...,144.0,175.0,271.0,0.576052,3317,3,0.474576,0.464286,0.600000,0.600000
3,3316,2379,2012,1,31,0.548822,8957,4,1,5,...,193.0,381.0,141.0,0.529183,3317,3,0.640000,0.285714,0.000000,0.000000
4,7869,6277,2012,1,31,0.378378,8957,4,1,5,...,215.0,352.0,152.0,0.419753,3317,3,0.400000,0.428571,0.500000,0.500000
5,7136,2107,2012,1,31,0.315789,8957,4,2,5,...,317.0,134.0,329.0,0.558559,3317,3,0.500000,0.357143,0.000000,0.200000
6,72,678,2012,1,31,0.487395,8887,4,2,5,...,51.0,1140.0,33.0,0.595890,3300,3,0.617647,0.482759,0.000000,0.333333
7,13796,563,2012,1,31,0.514286,8887,4,2,5,...,72.0,-9999.0,-9999.0,0.550877,3300,3,0.900000,0.468085,0.000000,0.416667
8,775,745,2012,1,31,0.532258,8888,4,2,5,...,198.0,547.0,105.0,0.570934,2460,3,0.425926,0.495652,0.250000,0.411765
9,75,1266,2012,1,31,0.556863,8888,4,1,5,...,39.0,710.0,67.0,0.547619,2460,3,0.551020,0.510000,0.500000,0.250000


## Pass Test : 

In [101]:
#df_merged_pos_tourney_test_J1
df_merged_column=df_merged_pos_tourney2.loc[:,['ID1_G','ID_T_REAL','WIN_FOR_TOURNEY_1']]
df_merged_column['ID1_G']=df_merged_column['ID1_G'].astype(int)
df_merged_test_final_test4_1_court=pd.merge(df_merged_test_final_test_court,df_merged_column,on=['ID1_G','ID_T_REAL'],how='left')
df_merged_test_final_test4_1_court=df_merged_test_final_test4_1_court.drop_duplicates(keep='last')
df_merged_test_final_test4_1_court

#df_merged_pos_tourney_test_J2
df_merged_column=df_merged_pos_tourney2.loc[:,['ID2_G','ID_T_REAL','WIN_FOR_TOURNEY_2']]
df_merged_column['ID2_G']=df_merged_column['ID2_G'].astype(int)

df_merged_test_final_test4_2_court=pd.merge(df_merged_test_final_test_court,df_merged_column,on=['ID2_G','ID_T_REAL'],how='left')
df_merged_test_final_test4_2_court=df_merged_test_final_test4_2_court.drop_duplicates(keep='last')
print(len(df_merged_test_final_test4_2_court))

df_merged_test_final_test_court_final=pd.merge(df_merged_test_final_test4_1_court,df_merged_test_final_test4_2_court, on=['ID1_G','ID2_G','YEAR','MONTH','DAY','WIN_POUR_1','WIN_POUR_2','POS_R_1','POS_R_2','POINT_R_1','POINT_R_2','ID_R_G','ID_T_G','ID_T_REAL','ID_C_T','WIN_COURT_1','WIN_COURT_2'],how='left')
df_merged_test_final_test_court_final=df_merged_test_final_test_court_final.fillna(-9999)

7608


In [102]:
df_merged_test_final_test_court_final

,ID1_G,ID2_G,YEAR,MONTH,DAY,WIN_POUR_1,ID_T_G,ID_R_G,POS_R_1,POS_R_2,POINT_R_1,POINT_R_2,WIN_POUR_2,ID_T_REAL,ID_C_T,WIN_COURT_1,WIN_COURT_2,WIN_FOR_TOURNEY_1,WIN_FOR_TOURNEY_2
0,30589,54796,2016,08,01,0.545455,13296,4,-9999,-9999,-9999,-9999,0.000000,4360,1,-9999.000000,-9999.000000,-9999.000000,-9999.000000
1,54409,24429,2016,08,01,0.500000,13296,4,-9999,-9999,-9999,-9999,0.628571,4360,1,-9999.000000,0.529412,-9999.000000,-9999.000000
2,28762,4025,2016,08,01,0.636364,13296,4,428,214,93,237,0.440191,4360,1,0.545455,0.538462,-9999.000000,-9999.000000
3,26009,19584,2016,08,01,0.441176,13296,4,617,288,50,173,0.475610,4360,1,0.490909,0.409836,-9999.000000,-9999.000000
4,12430,12767,2016,08,01,0.553191,13296,4,667,354,41,130,0.436170,4360,1,0.483871,0.521127,-9999.000000,-9999.000000
5,54797,24310,2016,08,01,0.000000,13296,4,-9999,-9999,-9999,-9999,0.423077,4360,1,-9999.000000,0.578947,-9999.000000,-9999.000000
6,54790,26730,2016,08,01,0.000000,13296,4,-9999,-9999,-9999,-9999,0.000000,4360,1,-9999.000000,-9999.000000,-9999.000000,-9999.000000
7,54792,54791,2016,08,01,0.000000,13296,4,-9999,-9999,-9999,-9999,0.000000,4360,1,-9999.000000,-9999.000000,-9999.000000,-9999.000000
8,11948,24531,2016,08,01,0.533333,13298,3,470,880,85,20,0.428571,3876,2,0.536585,0.640000,0.500000,0.750000
9,17928,36821,2016,08,01,0.546816,13298,3,138,791,431,26,0.555556,3876,2,0.482014,0.450000,0.571429,0.000000


In [ ]:
#X_test,Y_test=no_split_X_train(df_merged_test_final_test_court_final)

In [237]:
X_train,Y_train=no_split_X_train(df_merged_pos_tourney2)
X_test=df_merged_test_final_test_court_final
X_train=X_train.drop(['DATE_G_2'],axis=1)
Y_train.shape

(84485,)

In [104]:
X_train_h,Y_train_h,X_test_h,Y_test_h=prepare_database_hidden(X_train,Y_train)
print(len(Y_train_h)," ",Y_train_h.shape)
print(len(X_test_h)," ",X_test_h.shape)
print(len(Y_test_h)," ",Y_test_h.shape)
print(len(X_train_h)," ",X_train_h.shape)

62865   (62865,)
21619   (21619, 19)
21619   (21619,)
62865   (62865, 19)


In [105]:
Create_data_toevaluate(X_train_h,"X_train_4_pos_win_19f_sem_hidden",Y_train_h,"Y_train_4_pos_win_19f_sem_hidden",X_test_h,"X_test_4_pos_win_19f_sem_hidden",Y_test_h,"Y_test_4_pos_win_19f_sem_hidden")

In [238]:
Create_data_toevaluate(X_train,"X_train_4_pos_win_19f_sem",Y_train,"Y_train_4_pos_win_19f_sem",X_test,"X_test_4_pos_win_19f_sem",Y_test," ")

### CREATE FEATURE DATE & COUNTRY PLAYERS

In [122]:
temp=dict_table['players_atp']
temp = temp.loc[:,['ID_P', 'DATE_P','COUNTRY_P']]
temp.columns=['ID1_G','DATE_P_1','COUNTRY_P_1']
temp
df_merged_with_age_1 = pd.merge(df_merged_pos_tourney2,temp,on=['ID1_G'], how='left')
df_merged_with_age_1=df_merged_with_age_1.fillna(-9999)
df_merged_with_age_1['DATE_P_1']=df_merged_with_age_1['DATE_P_1'].astype('str')


temp=dict_table['players_atp']
temp = temp.loc[:,['ID_P', 'DATE_P','COUNTRY_P']]
temp.columns=['ID2_G','DATE_P_2','COUNTRY_P_2']
temp
df_merged_with_age_2 = pd.merge(df_merged_with_age_1,temp,on=['ID2_G'], how='left')
df_merged_with_age_2=df_merged_with_age_2.fillna(-9999)
df_merged_with_age_2['DATE_P_2']=df_merged_with_age_2['DATE_P_2'].astype('str')
df_merged_with_age_2=df_merged_with_age_2.drop_duplicates(keep='last')

len(df_merged_with_age_2)

84485

In [123]:
def parsedate_age_J1(row):
    datetime=row['DATE_P_1'].split('-')
    if(datetime[0]!='NaT' and datetime[0]!='NaN' and datetime[0]!='' ):
        row['YEAR_P_1']=datetime[0]
    else:
        row['YEAR_P_1']=-9999
    return row

In [124]:
def parsedate_age_J2(row):
    datetime=row['DATE_P_2'].split('-')
    if(datetime[0]!='NaT' and datetime[0]!='NaN' and datetime[0]!=''):
        row['YEAR_P_2']=datetime[0]
    else:
        row['YEAR_P_2']=-9999
    return row

In [125]:
df_merged_with_age_2=df_merged_with_age_2.apply(parsedate_age_J1,axis=1)
df_merged_with_age_2=df_merged_with_age_2.fillna(-9999)
df_merged_with_age_2['YEAR']=df_merged_with_age_2['YEAR'].astype('int')
df_merged_with_age_2['YEAR_P_1']=df_merged_with_age_2['YEAR_P_1'].astype('int')
df_merged_with_age_2['AGE_REAL_1']=df_merged_with_age_2['YEAR']-df_merged_with_age_2['YEAR_P_1']
df_merged_with_age_2=df_merged_with_age_2.drop_duplicates(keep='last')



df_merged_with_age_2=df_merged_with_age_2.apply(parsedate_age_J2,axis=1)
df_merged_with_age_2=df_merged_with_age_2.fillna(-9999)
df_merged_with_age_2['YEAR']=df_merged_with_age_2['YEAR'].astype('int')
df_merged_with_age_2['YEAR_P_2']=df_merged_with_age_2['YEAR_P_2'].astype('int')
df_merged_with_age_2['AGE_REAL_2']=df_merged_with_age_2['YEAR']-df_merged_with_age_2['YEAR_P_2']
df_merged_with_age_2=df_merged_with_age_2.drop_duplicates(keep='last')


df_merged_with_age_2=df_merged_with_age_2.drop(["YEAR_P_1"],axis=1)
df_merged_with_age_2=df_merged_with_age_2.drop(["YEAR_P_2"],axis=1)
df_merged_with_age_2=df_merged_with_age_2.drop(["DATE_P_1"],axis=1)
df_merged_with_age_2=df_merged_with_age_2.drop(["DATE_P_2"],axis=1)

df_merged_with_age_2=df_merged_with_age_2.fillna(-9999)

In [126]:
len(df_merged_with_age_2)

84485

In [127]:
for j in range(12011,12017):
        df_merged_with_age_2['AGE_REAL_1']=np.where(df_merged_with_age_2.AGE_REAL_1==j,-9999, df_merged_with_age_2['AGE_REAL_1'])
        df_merged_with_age_2['AGE_REAL_2']=np.where(df_merged_with_age_2.AGE_REAL_2==j,-9999, df_merged_with_age_2['AGE_REAL_2'])

print(len(df_merged_with_age_2[df_merged_with_age_2.AGE_REAL_1==-9999]))
print(len(df_merged_with_age_2[df_merged_with_age_2.AGE_REAL_2==-9999]))

14800
14815


In [128]:
df_merged_test_final_test_court_final['YEAR']=df_merged_test_final_test_court_final['YEAR'].astype('int')
df_merged_test_final_test_court_final
temp=df_merged_with_age_2
temp = temp.loc[:,['ID1_G', 'YEAR','COUNTRY_P_1','AGE_REAL_1']]
df_merged_with_age_1_test=pd.merge(df_merged_test_final_test_court_final,temp,on=['ID1_G','YEAR'],how='left')
df_merged_with_age_1_test=df_merged_with_age_1_test.drop_duplicates(keep='last')

print(len(df_merged_with_age_1_test))
temp2=df_merged_with_age_2
temp2 = temp2.loc[:,['ID2_G', 'YEAR','COUNTRY_P_2','AGE_REAL_2']]
df_merged_with_age_2_test=pd.merge(df_merged_with_age_1_test,temp2,on=['ID2_G','YEAR'],how='left')
df_merged_with_age_2_test=df_merged_with_age_2_test.fillna(-9999)
df_merged_with_age_2_test=df_merged_with_age_2_test.drop_duplicates(keep='last')
df_merged_with_age_2_test=df_merged_with_age_2_test.fillna(-9999)

7608


In [129]:
for j in range(12011,12017):
        df_merged_with_age_2_test['AGE_REAL_1']=np.where(df_merged_with_age_2_test.AGE_REAL_1==j,-9999, df_merged_with_age_2_test['AGE_REAL_1'])
        df_merged_with_age_2_test['AGE_REAL_2']=np.where(df_merged_with_age_2_test.AGE_REAL_2==j,-9999, df_merged_with_age_2_test['AGE_REAL_2'])

print(len(df_merged_with_age_2_test[df_merged_with_age_2_test.AGE_REAL_1==-9999]))
print(len(df_merged_with_age_2_test[df_merged_with_age_2_test.AGE_REAL_2==-9999]))

1354
1352


In [130]:
df_merged_with_age_2

,ID1_G,ID2_G,YEAR,MONTH,DAY,WIN_POUR_1,ID_T_G,ID_R_G,RESULT_G,DATE_G_2,...,ID_T_REAL,ID_C_T,WIN_COURT_1,WIN_COURT_2,WIN_FOR_TOURNEY_1,WIN_FOR_TOURNEY_2,COUNTRY_P_1,COUNTRY_P_2,AGE_REAL_1,AGE_REAL_2
0,17613,18854,2012,1,31,0.461538,8887,4,1,5,...,3300,3,0.428571,0.611111,0.500000,0.000000,-9999,-9999,-9999,-9999
1,25191,25192,2012,1,31,0.000000,8888,4,2,5,...,2460,3,-9999.000000,0.500000,0.000000,0.500000,-9999,-9999,-9999,-9999
2,918,10828,2012,1,31,0.558282,8957,4,1,5,...,3317,3,0.474576,0.464286,0.600000,0.600000,RUS,MDA,27,23
3,3316,2379,2012,1,31,0.548822,8957,4,1,5,...,3317,3,0.640000,0.285714,0.000000,0.000000,ESP,ESP,27,30
4,7869,6277,2012,1,31,0.378378,8957,4,1,5,...,3317,3,0.400000,0.428571,0.500000,0.500000,FIN,UKR,23,25
5,7136,2107,2012,1,31,0.315789,8957,4,2,5,...,3317,3,0.500000,0.357143,0.000000,0.200000,RUS,RUS,25,26
6,72,678,2012,1,31,0.487395,8887,4,2,5,...,3300,3,0.617647,0.482759,0.000000,0.333333,BEL,GER,31,29
7,13796,563,2012,1,31,0.514286,8887,4,2,5,...,3300,3,0.900000,0.468085,0.000000,0.416667,GER,FRA,22,30
8,775,745,2012,1,31,0.532258,8888,4,2,5,...,2460,3,0.425926,0.495652,0.250000,0.411765,SUI,GER,31,32
9,75,1266,2012,1,31,0.556863,8888,4,1,5,...,2460,3,0.551020,0.510000,0.500000,0.250000,RUS,UKR,30,26


In [131]:
X_train,Y_train=no_split_X_train(df_merged_with_age_2)
X_test=df_merged_with_age_2_test
X_test=X_test.drop('COUNTRY_P_1',axis=1)
X_test=X_test.drop('COUNTRY_P_2',axis=1)
X_train=X_train.drop('COUNTRY_P_1',axis=1)
X_train=X_train.drop('COUNTRY_P_2',axis=1)
X_train=X_train.drop('DATE_G_2',axis=1)


In [132]:
X_train_h,Y_train_h,X_test_h,Y_test_h=prepare_database_hidden(X_train,Y_train)
print(len(Y_train_h)," ",Y_train_h.shape)
print(len(X_test_h)," ",X_test_h.shape)
print(len(Y_test_h)," ",Y_test_h.shape)
print(len(X_train_h)," ",X_train_h.shape)

62865   (62865,)
21619   (21619, 21)
21619   (21619,)
62865   (62865, 21)


In [133]:
Create_data_toevaluate(X_train_h,"X_train_5_age_21f_sem_hidden",Y_train_h,"Y_train_5_age_21f_sem_hidden",X_test_h,"X_test_5_age_21f_sem_hidden",Y_test_h,"Y_test_5_age_21f_sem_hidden")

In [136]:
Y_test=0
Create_data_toevaluate(X_train,"X_train_5_age_21f_sem",Y_train,"Y_train_5_age_21f_sem",X_test,"X_test_5_age_21f_sem",Y_test," ")

## NEW FEATURE FACE TO FACE 

In [137]:
def face_to_face_win(table):
    for i in range(int(max(table.ID2_G))):
        if(len(table[table.ID1_G==i])!=0):
            tempbis=table[np.where((table.ID1_G==i) ,True,False)]
            for j in enumerate(tempbis['ID2_G']):
                id1=sum(np.where(tempbis.ID2_G==j[1],1,0))
                table['FACE_WIN_1']=np.where((table.ID1_G==i) & (table.ID2_G==j[1]),id1,table['FACE_WIN_1'])
                table['FACE_WIN_2']=np.where((table.ID2_G==i) & (table.ID1_G==j[1]),id1,table['FACE_WIN_2'])
    return table

In [138]:
debut=time.time()
table3=df_merged_with_age_2
table3['FACE_WIN_1']=0
table3['FACE_WIN_2']=0
#table3['ID1_G']=table3['ID1_G'].astype('int')
#table3['ID2_G']=table3['ID2_G'].astype('int')
table2=face_to_face_win(table3)
fin=time.time()
print("temps: ",fin-debut)
table2

temps:  289.8207378387451


,ID1_G,ID2_G,YEAR,MONTH,DAY,WIN_POUR_1,ID_T_G,ID_R_G,RESULT_G,DATE_G_2,...,WIN_COURT_1,WIN_COURT_2,WIN_FOR_TOURNEY_1,WIN_FOR_TOURNEY_2,COUNTRY_P_1,COUNTRY_P_2,AGE_REAL_1,AGE_REAL_2,FACE_WIN_1,FACE_WIN_2
0,17613,18854,2012,1,31,0.461538,8887,4,1,5,...,0.428571,0.611111,0.500000,0.000000,-9999,-9999,-9999,-9999,1,0
1,25191,25192,2012,1,31,0.000000,8888,4,2,5,...,-9999.000000,0.500000,0.000000,0.500000,-9999,-9999,-9999,-9999,1,0
2,918,10828,2012,1,31,0.558282,8957,4,1,5,...,0.474576,0.464286,0.600000,0.600000,RUS,MDA,27,23,3,2
3,3316,2379,2012,1,31,0.548822,8957,4,1,5,...,0.640000,0.285714,0.000000,0.000000,ESP,ESP,27,30,1,0
4,7869,6277,2012,1,31,0.378378,8957,4,1,5,...,0.400000,0.428571,0.500000,0.500000,FIN,UKR,23,25,1,0
5,7136,2107,2012,1,31,0.315789,8957,4,2,5,...,0.500000,0.357143,0.000000,0.200000,RUS,RUS,25,26,1,0
6,72,678,2012,1,31,0.487395,8887,4,2,5,...,0.617647,0.482759,0.000000,0.333333,BEL,GER,31,29,3,0
7,13796,563,2012,1,31,0.514286,8887,4,2,5,...,0.900000,0.468085,0.000000,0.416667,GER,FRA,22,30,1,0
8,775,745,2012,1,31,0.532258,8888,4,2,5,...,0.425926,0.495652,0.250000,0.411765,SUI,GER,31,32,2,1
9,75,1266,2012,1,31,0.556863,8888,4,1,5,...,0.551020,0.510000,0.500000,0.250000,RUS,UKR,30,26,2,1


In [140]:
#df_merge_fight_J1
df_merged_column=table2.loc[:,['ID1_G','ID2_G','FACE_WIN_1']]
df_merged_column['ID1_G']=df_merged_column['ID1_G'].astype(int)
df_merged_test_final_fight_1=pd.merge(df_merged_with_age_2_test,df_merged_column,on=['ID1_G','ID2_G'],how='left')
df_merged_test_final_fight_1=df_merged_test_final_fight_1.drop_duplicates(keep='last')
print(len(df_merged_test_final_fight_1))

#df_merged_fight_J2
df_merged_column=table2.loc[:,['ID1_G','ID2_G','FACE_WIN_2',]]
df_merged_column['ID2_G']=df_merged_column['ID2_G'].astype(int)

df_merged_test_final_fight_2=pd.merge(df_merged_with_age_2_test,df_merged_column,on=['ID1_G','ID2_G'],how='left')
df_merged_test_final_fight_2=df_merged_test_final_fight_2.drop_duplicates(keep='last')
print(len(df_merged_test_final_fight_2))

df_merged_test_final_test_fight=pd.merge(df_merged_test_final_fight_1,df_merged_test_final_fight_2, on=['ID1_G','ID2_G','YEAR','MONTH','DAY','WIN_POUR_1','WIN_POUR_2','POS_R_1','POS_R_2','ID_R_G','ID_T_G','ID_T_REAL','WIN_FOR_TOURNEY_1','WIN_FOR_TOURNEY_2','COUNTRY_P_1','COUNTRY_P_2','POINT_R_1','POINT_R_2','WIN_COURT_1','WIN_COURT_2','AGE_REAL_1','AGE_REAL_2','ID_C_T'],how='left')
df_merged_test_final_test_fight

7608
7608


,ID1_G,ID2_G,YEAR,MONTH,DAY,WIN_POUR_1,ID_T_G,ID_R_G,POS_R_1,POS_R_2,...,WIN_COURT_1,WIN_COURT_2,WIN_FOR_TOURNEY_1,WIN_FOR_TOURNEY_2,COUNTRY_P_1,AGE_REAL_1,COUNTRY_P_2,AGE_REAL_2,FACE_WIN_1,FACE_WIN_2
0,30589,54796,2016,08,01,0.545455,13296,4,-9999,-9999,...,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999,-9999.0,-9999,-9999.0,NaN,NaN
1,54409,24429,2016,08,01,0.500000,13296,4,-9999,-9999,...,-9999.000000,0.529412,-9999.000000,-9999.000000,-9999,-9999.0,-9999,-9999.0,1.0,0.0
2,28762,4025,2016,08,01,0.636364,13296,4,428,214,...,0.545455,0.538462,-9999.000000,-9999.000000,KOR,19.0,TPE,33.0,1.0,0.0
3,26009,19584,2016,08,01,0.441176,13296,4,617,288,...,0.490909,0.409836,-9999.000000,-9999.000000,CHN,22.0,COL,29.0,NaN,NaN
4,12430,12767,2016,08,01,0.553191,13296,4,667,354,...,0.483871,0.521127,-9999.000000,-9999.000000,IND,27.0,JPN,29.0,NaN,NaN
5,54797,24310,2016,08,01,0.000000,13296,4,-9999,-9999,...,-9999.000000,0.578947,-9999.000000,-9999.000000,-9999,-9999.0,-9999,-9999.0,NaN,NaN
6,54790,26730,2016,08,01,0.000000,13296,4,-9999,-9999,...,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999,-9999.0,-9999,-9999.0,NaN,NaN
7,54792,54791,2016,08,01,0.000000,13296,4,-9999,-9999,...,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999,-9999.0,-9999,-9999.0,NaN,NaN
8,11948,24531,2016,08,01,0.533333,13298,3,470,880,...,0.536585,0.640000,0.500000,0.750000,SWE,27.0,CZE,26.0,NaN,NaN
9,17928,36821,2016,08,01,0.546816,13298,3,138,791,...,0.482014,0.450000,0.571429,0.000000,SVK,26.0,SWE,19.0,NaN,NaN


### Partie Test : 

In [142]:
df_merged_test_final_test_fight=df_merged_test_final_test_fight.fillna(-9999)

X_train,Y_train=no_split_X_train(table2)
X_test=df_merged_test_final_test_fight
X_test=X_test.drop('COUNTRY_P_1',axis=1)
X_test=X_test.drop('COUNTRY_P_2',axis=1)
X_train=X_train.drop('COUNTRY_P_1',axis=1)
X_train=X_train.drop('COUNTRY_P_2',axis=1)
X_train=X_train.drop('DATE_G_2',axis=1)

X_test

,ID1_G,ID2_G,YEAR,MONTH,DAY,WIN_POUR_1,ID_T_G,ID_R_G,POS_R_1,POS_R_2,...,ID_T_REAL,ID_C_T,WIN_COURT_1,WIN_COURT_2,WIN_FOR_TOURNEY_1,WIN_FOR_TOURNEY_2,AGE_REAL_1,AGE_REAL_2,FACE_WIN_1,FACE_WIN_2
0,30589,54796,2016,08,01,0.545455,13296,4,-9999,-9999,...,4360,1,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.0,-9999.0,-9999.0,-9999.0
1,54409,24429,2016,08,01,0.500000,13296,4,-9999,-9999,...,4360,1,-9999.000000,0.529412,-9999.000000,-9999.000000,-9999.0,-9999.0,1.0,0.0
2,28762,4025,2016,08,01,0.636364,13296,4,428,214,...,4360,1,0.545455,0.538462,-9999.000000,-9999.000000,19.0,33.0,1.0,0.0
3,26009,19584,2016,08,01,0.441176,13296,4,617,288,...,4360,1,0.490909,0.409836,-9999.000000,-9999.000000,22.0,29.0,-9999.0,-9999.0
4,12430,12767,2016,08,01,0.553191,13296,4,667,354,...,4360,1,0.483871,0.521127,-9999.000000,-9999.000000,27.0,29.0,-9999.0,-9999.0
5,54797,24310,2016,08,01,0.000000,13296,4,-9999,-9999,...,4360,1,-9999.000000,0.578947,-9999.000000,-9999.000000,-9999.0,-9999.0,-9999.0,-9999.0
6,54790,26730,2016,08,01,0.000000,13296,4,-9999,-9999,...,4360,1,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.0,-9999.0,-9999.0,-9999.0
7,54792,54791,2016,08,01,0.000000,13296,4,-9999,-9999,...,4360,1,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.0,-9999.0,-9999.0,-9999.0
8,11948,24531,2016,08,01,0.533333,13298,3,470,880,...,3876,2,0.536585,0.640000,0.500000,0.750000,27.0,26.0,-9999.0,-9999.0
9,17928,36821,2016,08,01,0.546816,13298,3,138,791,...,3876,2,0.482014,0.450000,0.571429,0.000000,26.0,19.0,-9999.0,-9999.0


In [146]:
X_train_h,Y_train_h,X_test_h,Y_test_h=prepare_database_hidden(X_train,Y_train)
print(len(Y_train_h)," ",Y_train_h.shape)
print(len(X_test_h)," ",X_test_h.shape)
print(len(Y_test_h)," ",Y_test_h.shape)
print(len(X_train_h)," ",X_train_h.shape)

62865   (62865,)
21619   (21619, 23)
21619   (21619,)
62865   (62865, 23)


In [164]:
len(X_test[X_test.FACE_WIN_1==-9999])

6957

In [165]:
Create_data_toevaluate(X_train_h,"X_train_6_face_23f_sem_hidden",Y_train_h,"Y_train_6_face_23f_sem_hidden",X_test_h,"X_test_6_face_23f_sem_hidden",Y_test_h,"Y_test_6_face_23f_sem_hidden")

In [ ]:
Create_data_toevaluate(X_train,"X_train_6_face_23f_sem",Y_train,"Y_train_6_face_23f_sem",X_test,"X_test_6_face_23f_sem",Y_test," ")

In [145]:
X_train

,ID1_G,ID2_G,YEAR,MONTH,DAY,WIN_POUR_1,ID_T_G,ID_R_G,POINT_R_1,POS_R_1,...,ID_T_REAL,ID_C_T,WIN_COURT_1,WIN_COURT_2,WIN_FOR_TOURNEY_1,WIN_FOR_TOURNEY_2,AGE_REAL_1,AGE_REAL_2,FACE_WIN_1,FACE_WIN_2
0,17613,18854,2012,1,31,0.461538,8887,4,-9999.0,-9999.0,...,3300,3,0.428571,0.611111,0.500000,0.000000,-9999,-9999,1,0
1,25191,25192,2012,1,31,0.000000,8888,4,-9999.0,-9999.0,...,2460,3,-9999.000000,0.500000,0.000000,0.500000,-9999,-9999,1,0
2,918,10828,2012,1,31,0.558282,8957,4,375.0,144.0,...,3317,3,0.474576,0.464286,0.600000,0.600000,27,23,3,2
3,3316,2379,2012,1,31,0.548822,8957,4,254.0,193.0,...,3317,3,0.640000,0.285714,0.000000,0.000000,27,30,1,0
4,7869,6277,2012,1,31,0.378378,8957,4,223.0,215.0,...,3317,3,0.400000,0.428571,0.500000,0.500000,23,25,1,0
5,7136,2107,2012,1,31,0.315789,8957,4,140.0,317.0,...,3317,3,0.500000,0.357143,0.000000,0.200000,25,26,1,0
6,72,678,2012,1,31,0.487395,8887,4,899.0,51.0,...,3300,3,0.617647,0.482759,0.000000,0.333333,31,29,3,0
7,13796,563,2012,1,31,0.514286,8887,4,688.0,72.0,...,3300,3,0.900000,0.468085,0.000000,0.416667,22,30,1,0
8,775,745,2012,1,31,0.532258,8888,4,243.0,198.0,...,2460,3,0.425926,0.495652,0.250000,0.411765,31,32,2,1
9,75,1266,2012,1,31,0.556863,8888,4,1025.0,39.0,...,2460,3,0.551020,0.510000,0.500000,0.250000,30,26,2,1


## NEW FEATURE COUNTRY TOURNEY AND COUNTRY PEOPLE: 

In [166]:
for i in enumerate(dict_table['players_atp']['COUNTRY_P'].unique()):
    table2['COUNTRY_P_1']=np.where(table2['COUNTRY_P_1']==i[1],i[0],table2['COUNTRY_P_1'])
    table2['COUNTRY_P_2']=np.where(table2['COUNTRY_P_2']==i[1],i[0],table2['COUNTRY_P_2'])
table2

,ID1_G,ID2_G,YEAR,MONTH,DAY,WIN_POUR_1,ID_T_G,ID_R_G,RESULT_G,DATE_G_2,...,WIN_COURT_1,WIN_COURT_2,WIN_FOR_TOURNEY_1,WIN_FOR_TOURNEY_2,COUNTRY_P_1,COUNTRY_P_2,AGE_REAL_1,AGE_REAL_2,FACE_WIN_1,FACE_WIN_2
0,17613,18854,2012,1,31,0.461538,8887,4,1,5,...,0.428571,0.611111,0.500000,0.000000,-9999,-9999,-9999,-9999,1,0
1,25191,25192,2012,1,31,0.000000,8888,4,2,5,...,-9999.000000,0.500000,0.000000,0.500000,-9999,-9999,-9999,-9999,1,0
2,918,10828,2012,1,31,0.558282,8957,4,1,5,...,0.474576,0.464286,0.600000,0.600000,1,88,27,23,3,2
3,3316,2379,2012,1,31,0.548822,8957,4,1,5,...,0.640000,0.285714,0.000000,0.000000,6,6,27,30,1,0
4,7869,6277,2012,1,31,0.378378,8957,4,1,5,...,0.400000,0.428571,0.500000,0.500000,45,22,23,25,1,0
5,7136,2107,2012,1,31,0.315789,8957,4,2,5,...,0.500000,0.357143,0.000000,0.200000,1,1,25,26,1,0
6,72,678,2012,1,31,0.487395,8887,4,2,5,...,0.617647,0.482759,0.000000,0.333333,25,13,31,29,3,0
7,13796,563,2012,1,31,0.514286,8887,4,2,5,...,0.900000,0.468085,0.000000,0.416667,13,7,22,30,1,0
8,775,745,2012,1,31,0.532258,8888,4,2,5,...,0.425926,0.495652,0.250000,0.411765,10,13,31,32,2,1
9,75,1266,2012,1,31,0.556863,8888,4,1,5,...,0.551020,0.510000,0.500000,0.250000,1,22,30,26,2,1


In [167]:
table2

,ID1_G,ID2_G,YEAR,MONTH,DAY,WIN_POUR_1,ID_T_G,ID_R_G,RESULT_G,DATE_G_2,...,WIN_COURT_1,WIN_COURT_2,WIN_FOR_TOURNEY_1,WIN_FOR_TOURNEY_2,COUNTRY_P_1,COUNTRY_P_2,AGE_REAL_1,AGE_REAL_2,FACE_WIN_1,FACE_WIN_2
0,17613,18854,2012,1,31,0.461538,8887,4,1,5,...,0.428571,0.611111,0.500000,0.000000,-9999,-9999,-9999,-9999,1,0
1,25191,25192,2012,1,31,0.000000,8888,4,2,5,...,-9999.000000,0.500000,0.000000,0.500000,-9999,-9999,-9999,-9999,1,0
2,918,10828,2012,1,31,0.558282,8957,4,1,5,...,0.474576,0.464286,0.600000,0.600000,1,88,27,23,3,2
3,3316,2379,2012,1,31,0.548822,8957,4,1,5,...,0.640000,0.285714,0.000000,0.000000,6,6,27,30,1,0
4,7869,6277,2012,1,31,0.378378,8957,4,1,5,...,0.400000,0.428571,0.500000,0.500000,45,22,23,25,1,0
5,7136,2107,2012,1,31,0.315789,8957,4,2,5,...,0.500000,0.357143,0.000000,0.200000,1,1,25,26,1,0
6,72,678,2012,1,31,0.487395,8887,4,2,5,...,0.617647,0.482759,0.000000,0.333333,25,13,31,29,3,0
7,13796,563,2012,1,31,0.514286,8887,4,2,5,...,0.900000,0.468085,0.000000,0.416667,13,7,22,30,1,0
8,775,745,2012,1,31,0.532258,8888,4,2,5,...,0.425926,0.495652,0.250000,0.411765,10,13,31,32,2,1
9,75,1266,2012,1,31,0.556863,8888,4,1,5,...,0.551020,0.510000,0.500000,0.250000,1,22,30,26,2,1


In [168]:
for i in enumerate(dict_table['players_atp']['COUNTRY_P'].unique()):
    df_merged_test_final_test_fight['COUNTRY_P_1']=np.where(df_merged_test_final_test_fight['COUNTRY_P_1']==i[1],i[0],df_merged_test_final_test_fight['COUNTRY_P_1'])
    df_merged_test_final_test_fight['COUNTRY_P_2']=np.where(df_merged_test_final_test_fight['COUNTRY_P_2']==i[1],i[0],df_merged_test_final_test_fight['COUNTRY_P_2'])
df_merged_test_final_test_fight

,ID1_G,ID2_G,YEAR,MONTH,DAY,WIN_POUR_1,ID_T_G,ID_R_G,POS_R_1,POS_R_2,...,WIN_COURT_1,WIN_COURT_2,WIN_FOR_TOURNEY_1,WIN_FOR_TOURNEY_2,COUNTRY_P_1,AGE_REAL_1,COUNTRY_P_2,AGE_REAL_2,FACE_WIN_1,FACE_WIN_2
0,30589,54796,2016,08,01,0.545455,13296,4,-9999,-9999,...,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999,-9999.0,-9999,-9999.0,-9999.0,-9999.0
1,54409,24429,2016,08,01,0.500000,13296,4,-9999,-9999,...,-9999.000000,0.529412,-9999.000000,-9999.000000,-9999,-9999.0,-9999,-9999.0,1.0,0.0
2,28762,4025,2016,08,01,0.636364,13296,4,428,214,...,0.545455,0.538462,-9999.000000,-9999.000000,26,19.0,70,33.0,1.0,0.0
3,26009,19584,2016,08,01,0.441176,13296,4,617,288,...,0.490909,0.409836,-9999.000000,-9999.000000,52,22.0,47,29.0,-9999.0,-9999.0
4,12430,12767,2016,08,01,0.553191,13296,4,667,354,...,0.483871,0.521127,-9999.000000,-9999.000000,35,27.0,41,29.0,-9999.0,-9999.0
5,54797,24310,2016,08,01,0.000000,13296,4,-9999,-9999,...,-9999.000000,0.578947,-9999.000000,-9999.000000,-9999,-9999.0,-9999,-9999.0,-9999.0,-9999.0
6,54790,26730,2016,08,01,0.000000,13296,4,-9999,-9999,...,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999,-9999.0,-9999,-9999.0,-9999.0,-9999.0
7,54792,54791,2016,08,01,0.000000,13296,4,-9999,-9999,...,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999,-9999.0,-9999,-9999.0,-9999.0,-9999.0
8,11948,24531,2016,08,01,0.533333,13298,3,470,880,...,0.536585,0.640000,0.500000,0.750000,3,27.0,19,26.0,-9999.0,-9999.0
9,17928,36821,2016,08,01,0.546816,13298,3,138,791,...,0.482014,0.450000,0.571429,0.000000,9,26.0,3,19.0,-9999.0,-9999.0


In [169]:
table_final=table2
temp=dict_table['tours_atp']
temp = temp.loc[:,['ID_T','COUNTRY_T']]
temp.columns=['ID_T_G','COUNTRY_T']
table_final_2=pd.merge(table_final,temp,on=['ID_T_G'],how='left')
for i in enumerate(dict_table['players_atp']['COUNTRY_P'].unique()):
    table_final_2['COUNTRY_T']=np.where(table_final_2['COUNTRY_T']==i[1],i[0],table_final_2['COUNTRY_T'])
table_final_2

,ID1_G,ID2_G,YEAR,MONTH,DAY,WIN_POUR_1,ID_T_G,ID_R_G,RESULT_G,DATE_G_2,...,WIN_COURT_2,WIN_FOR_TOURNEY_1,WIN_FOR_TOURNEY_2,COUNTRY_P_1,COUNTRY_P_2,AGE_REAL_1,AGE_REAL_2,FACE_WIN_1,FACE_WIN_2,COUNTRY_T
0,17613,18854,2012,1,31,0.461538,8887,4,1,5,...,0.611111,0.500000,0.000000,-9999,-9999,-9999,-9999,1,0,7
1,25191,25192,2012,1,31,0.000000,8888,4,2,5,...,0.500000,0.000000,0.500000,-9999,-9999,-9999,-9999,1,0,24
2,918,10828,2012,1,31,0.558282,8957,4,1,5,...,0.464286,0.600000,0.600000,1,88,27,23,3,2,1
3,3316,2379,2012,1,31,0.548822,8957,4,1,5,...,0.285714,0.000000,0.000000,6,6,27,30,1,0,1
4,7869,6277,2012,1,31,0.378378,8957,4,1,5,...,0.428571,0.500000,0.500000,45,22,23,25,1,0,1
5,7136,2107,2012,1,31,0.315789,8957,4,2,5,...,0.357143,0.000000,0.200000,1,1,25,26,1,0,1
6,72,678,2012,1,31,0.487395,8887,4,2,5,...,0.482759,0.000000,0.333333,25,13,31,29,3,0,7
7,13796,563,2012,1,31,0.514286,8887,4,2,5,...,0.468085,0.000000,0.416667,13,7,22,30,1,0,7
8,775,745,2012,1,31,0.532258,8888,4,2,5,...,0.495652,0.250000,0.411765,10,13,31,32,2,1,24
9,75,1266,2012,1,31,0.556863,8888,4,1,5,...,0.510000,0.500000,0.250000,1,22,30,26,2,1,24


In [170]:
table_final_test=df_merged_test_final_test_fight
temp=dict_table['tours_atp']
temp = temp.loc[:,['ID_T','COUNTRY_T']]
temp.columns=['ID_T_G','COUNTRY_T']
table_final_2_test=pd.merge(table_final_test,temp,on=['ID_T_G'],how='left')
for i in enumerate(dict_table['players_atp']['COUNTRY_P'].unique()):
    table_final_2_test['COUNTRY_T']=np.where(table_final_2_test['COUNTRY_T']==i[1],i[0],table_final_2_test['COUNTRY_T'])
table_final_2_test

,ID1_G,ID2_G,YEAR,MONTH,DAY,WIN_POUR_1,ID_T_G,ID_R_G,POS_R_1,POS_R_2,...,WIN_COURT_2,WIN_FOR_TOURNEY_1,WIN_FOR_TOURNEY_2,COUNTRY_P_1,AGE_REAL_1,COUNTRY_P_2,AGE_REAL_2,FACE_WIN_1,FACE_WIN_2,COUNTRY_T
0,30589,54796,2016,08,01,0.545455,13296,4,-9999,-9999,...,-9999.000000,-9999.000000,-9999.000000,-9999,-9999.0,-9999,-9999.0,-9999.0,-9999.0,52
1,54409,24429,2016,08,01,0.500000,13296,4,-9999,-9999,...,0.529412,-9999.000000,-9999.000000,-9999,-9999.0,-9999,-9999.0,1.0,0.0,52
2,28762,4025,2016,08,01,0.636364,13296,4,428,214,...,0.538462,-9999.000000,-9999.000000,26,19.0,70,33.0,1.0,0.0,52
3,26009,19584,2016,08,01,0.441176,13296,4,617,288,...,0.409836,-9999.000000,-9999.000000,52,22.0,47,29.0,-9999.0,-9999.0,52
4,12430,12767,2016,08,01,0.553191,13296,4,667,354,...,0.521127,-9999.000000,-9999.000000,35,27.0,41,29.0,-9999.0,-9999.0,52
5,54797,24310,2016,08,01,0.000000,13296,4,-9999,-9999,...,0.578947,-9999.000000,-9999.000000,-9999,-9999.0,-9999,-9999.0,-9999.0,-9999.0,52
6,54790,26730,2016,08,01,0.000000,13296,4,-9999,-9999,...,-9999.000000,-9999.000000,-9999.000000,-9999,-9999.0,-9999,-9999.0,-9999.0,-9999.0,52
7,54792,54791,2016,08,01,0.000000,13296,4,-9999,-9999,...,-9999.000000,-9999.000000,-9999.000000,-9999,-9999.0,-9999,-9999.0,-9999.0,-9999.0,52
8,11948,24531,2016,08,01,0.533333,13298,3,470,880,...,0.640000,0.500000,0.750000,3,27.0,19,26.0,-9999.0,-9999.0,19
9,17928,36821,2016,08,01,0.546816,13298,3,138,791,...,0.450000,0.571429,0.000000,9,26.0,3,19.0,-9999.0,-9999.0,19


In [285]:
table_final_2_test=table_final_2_test.fillna(-9999)
X_test=table_final_2_test
table_final_2=table_final_2.fillna(-9999)
X_train,Y_train=no_split_X_train(table_final_2)
X_train=X_train.drop('DATE_G_2',axis=1)

In [172]:
X_train_h,Y_train_h,X_test_h,Y_test_h=prepare_database_hidden(X_train,Y_train)
print(len(Y_train_h)," ",Y_train_h.shape)
print(len(X_test_h)," ",X_test_h.shape)
print(len(Y_test_h)," ",Y_test_h.shape)
print(len(X_train_h)," ",X_train_h.shape)

62865   (62865,)
21619   (21619, 26)
21619   (21619,)
62865   (62865, 26)


In [174]:
Create_data_toevaluate(X_train_h,"X_train_final_26f_sem2_hidden",Y_train_h,"Y_train_final_26f_sem2_hidden",X_test_h,"X_test_final_26f_sem2_hidden",Y_test_h,"Y_test_final_26f_sem2_hidden")

In [173]:
Create_data_toevaluate(X_train,"X_train_final_26f_sem2",Y_train,"Y_train_final_26f_sem2",X_test,"X_test_final_26f_sem2",Y_test," ")

In [176]:
X_train
X_train=X_train.replace([-9999],[-1])


In [177]:
X_train_h,Y_train_h,X_test_h,Y_test_h=prepare_database_hidden(X_train,Y_train)
print(len(Y_train_h)," ",Y_train_h.shape)
print(len(X_test_h)," ",X_test_h.shape)
print(len(Y_test_h)," ",Y_test_h.shape)
print(len(X_train_h)," ",X_train_h.shape)

62865   (62865,)
21619   (21619, 26)
21619   (21619,)
62865   (62865, 26)


In [178]:
Create_data_toevaluate(X_train_h,"X_train_final_26f_sem2_hidden_by1",Y_train_h,"Y_train_final_26f_sem2_hidden_by1",X_test_h,"X_test_final_26f_sem2_hidden_by1",Y_test_h,"Y_test_final_26f_sem2_hidden_by1")

In [282]:
X_train.columns

Index(['ID1_G', 'ID2_G', 'YEAR', 'MONTH', 'DAY', 'WIN_POUR_1', 'ID_T_G',
       'ID_R_G', 'POINT_R_1', 'POS_R_1', 'POINT_R_2', 'POS_R_2', 'WIN_POUR_2',
       'ID_T_REAL', 'ID_C_T', 'WIN_COURT_1', 'WIN_COURT_2',
       'WIN_FOR_TOURNEY_1', 'WIN_FOR_TOURNEY_2', 'COUNTRY_P_1', 'COUNTRY_P_2',
       'AGE_REAL_1', 'AGE_REAL_2', 'FACE_WIN_1', 'FACE_WIN_2', 'COUNTRY_T'],
      dtype='object')

In [290]:
X_train_RFE=X_train.drop('ID_C_T',axis=1)
X_train_RFE=X_train_RFE.drop('MONTH',axis=1)
X_train_RFE=X_train_RFE.drop('DAY',axis=1)
X_train_RFE=X_train_RFE.drop('ID_T_G',axis=1)
#X_train_RFE=X_train_RFE.drop('POINT_R_1',axis=1)
#X_train_RFE=X_train_RFE.drop('POINT_R_2',axis=1)
#X_train_RFE=X_train_RFE.drop('POS_R_1',axis=1)
#X_train_RFE=X_train_RFE.drop('POS_R_2',axis=1)
X_train_RFE=X_train_RFE.drop('YEAR',axis=1)
X_train_RFE=X_train_RFE.drop('AGE_REAL_1',axis=1)
X_train_RFE=X_train_RFE.drop('WIN_POUR_1',axis=1)
X_train_RFE=X_train_RFE.drop('WIN_POUR_2',axis=1)
X_train_RFE=X_train_RFE.drop('WIN_COURT_2',axis=1)
X_train_RFE=X_train_RFE.drop('WIN_COURT_1',axis=1)
X_train_RFE=X_train_RFE.drop('FACE_WIN_1',axis=1)
X_train_RFE=X_train_RFE.drop('FACE_WIN_2',axis=1)
X_train_RFE=X_train_RFE.drop('WIN_FOR_TOURNEY_2',axis=1)
X_train_RFE=X_train_RFE.drop('WIN_FOR_TOURNEY_1',axis=1)
X_train_RFE=X_train_RFE.drop('AGE_REAL_2',axis=1)
X_train_RFE=X_train_RFE.drop('COUNTRY_P_1',axis=1)
X_train_RFE=X_train_RFE.drop('COUNTRY_P_2',axis=1)
X_train_RFE=X_train_RFE.drop('COUNTRY_T',axis=1)
X_train_RFE=X_train_RFE.drop('ID_T_REAL',axis=1)


X_test_final=table_final_2_test.drop('ID_C_T',axis=1)
X_test_final=X_test_final.drop('MONTH',axis=1)
X_test_final=X_test_final.drop('DAY',axis=1)
X_test_final=X_test_final.drop('ID_T_G',axis=1)
#X_test_final=X_test_final.drop('POINT_R_1',axis=1)
#X_test_final=X_test_final.drop('POINT_R_2',axis=1)
#X_test_final=X_test_final.drop('POS_R_1',axis=1)
#X_test_final=X_test_final.drop('POS_R_2',axis=1)
X_test_final=X_test_final.drop('YEAR',axis=1)
X_test_final=X_test_final.drop('AGE_REAL_1',axis=1)
X_test_final=X_test_final.drop('WIN_POUR_1',axis=1)
X_test_final=X_test_final.drop('WIN_POUR_2',axis=1)
X_test_final=X_test_final.drop('WIN_COURT_2',axis=1)
X_test_final=X_test_final.drop('WIN_COURT_1',axis=1)
X_test_final=X_test_final.drop('FACE_WIN_1',axis=1)
X_test_final=X_test_final.drop('FACE_WIN_2',axis=1)
X_test_final=X_test_final.drop('WIN_FOR_TOURNEY_2',axis=1)
X_test_final=X_test_final.drop('WIN_FOR_TOURNEY_1',axis=1)
X_test_final=X_test_final.drop('AGE_REAL_2',axis=1)
X_test_final=X_test_final.drop('COUNTRY_T',axis=1)
X_test_final=X_test_final.drop('COUNTRY_P_1',axis=1)
X_test_final=X_test_final.drop('COUNTRY_P_2',axis=1)
X_test_final=X_test_final.drop('ID_T_REAL',axis=1)



In [206]:
X_train_RFE.to_csv('train_test_csv3/final_features.csv',header=True, index=None)

In [207]:
X_train_h,Y_train_h,X_test_h,Y_test_h=prepare_database_hidden(X_train_RFE,Y_train)
print(len(Y_train_h)," ",Y_train_h.shape)
print(len(X_test_h)," ",X_test_h.shape)
print(len(Y_test_h)," ",Y_test_h.shape)
print(len(X_train_h)," ",X_train_h.shape)

62865   (62865,)
21619   (21619, 11)
21619   (21619,)
62865   (62865, 11)


In [210]:
Create_data_toevaluate(X_train_h,"X_train_with_all_ultime_drop_2_hidden",Y_train_h,"Y_train_with_all_ultime_drop_2_hidden",X_test_h,"X_test_with_all_ultime_drop_2_hidden",Y_test_h,"Y_test_with_all_ultime_drop_2_hidden")

In [291]:
Create_data_toevaluate(X_train_RFE,"X_train_with_all_ultime_drop_2_final",Y_train,"Y_train_with_all_ultime_drop_2_final",X_test_final,"X_test_with_all_ultime_drop_2_final",Y_test," ")

In [ ]:
X_test_final.columns

In [ ]:
X_train_RFE.columns

In [ ]:
X_test_final

In [ ]:
X_test

In [198]:
X_train_RFE

,ID1_G,ID2_G,ID_R_G,POINT_R_1,POS_R_1,POINT_R_2,POS_R_2,ID_T_REAL,AGE_REAL_1,AGE_REAL_2
0,17613,18854,4,-1.0,-1.0,-1.0,-1.0,3300,-1,-1
1,25191,25192,4,-1.0,-1.0,-1.0,-1.0,2460,-1,-1
2,918,10828,4,375.0,144.0,175.0,271.0,3317,27,23
3,3316,2379,4,254.0,193.0,381.0,141.0,3317,27,30
4,7869,6277,4,223.0,215.0,352.0,152.0,3317,23,25
5,7136,2107,4,140.0,317.0,134.0,329.0,3317,25,26
6,72,678,4,899.0,51.0,1140.0,33.0,3300,31,29
7,13796,563,4,688.0,72.0,-1.0,-1.0,3300,22,30
8,775,745,4,243.0,198.0,547.0,105.0,2460,31,32
9,75,1266,4,1025.0,39.0,710.0,67.0,2460,30,26


In [215]:
X_test_final2=X_test_final
X_train_RFE_2=X_train_RFE
X_test_final2['AGE_REAL_1']=np.where(X_test_final2.POS_R_1!=-1,-1,X_test_final2['AGE_REAL_1'])
X_test_final2['AGE_REAL_2']=np.where(X_test_final2.POS_R_2!=-1,-1,X_test_final2['AGE_REAL_2'])
X_train_RFE_2['AGE_REAL_2']=np.where(X_train_RFE_2.POS_R_2!=-1,-1,X_train_RFE_2['AGE_REAL_2'])
X_train_RFE_2['AGE_REAL_1']=np.where(X_train_RFE_2.POS_R_1!=-1,-1,X_train_RFE_2['AGE_REAL_1'])

    #if(np.where(X_test_final2.POS_R_2!=-9999,True,False)):
    #    X_test_final2['AGE_REAL_2']=-9999

In [216]:
X_train_h,Y_train_h,X_test_h,Y_test_h=prepare_database_hidden(X_train_RFE_2,Y_train)
print(len(Y_train_h)," ",Y_train_h.shape)
print(len(X_test_h)," ",X_test_h.shape)
print(len(Y_test_h)," ",Y_test_h.shape)
print(len(X_train_h)," ",X_train_h.shape)

Create_data_toevaluate(X_train_h,"X_train_with_all_ultime_drop_2_age_hidden",Y_train_h,"Y_train_with_all_ultime_drop_2_age_hidden",X_test_h,"X_test_with_all_ultime_drop_2_age_hidden",Y_test_h,"Y_test_with_all_ultime_drop_2_age_hidden")

62865   (62865,)
21619   (21619, 11)
21619   (21619,)
62865   (62865, 11)


In [212]:
Create_data_toevaluate(X_train_RFE_2,"X_train_with_all_ultime_drop_2_age",Y_train,"Y_train_with_all_ultime_drop_2_age",X_test_final2,"X_test_with_all_ultime_drop_2_age",Y_test," ")